<a href="https://colab.research.google.com/github/yi-juho/-K_MHAS-_HuggingFace_dataset_MultiBERT_shared/blob/main/%5BK_MHAS%5D_HuggingFace_dataset_MultiBERT_shared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K-MHaS (Korean Multi-label Hate Speech Dataset)

## Dataset loading

Loading the K-MHaS dataset from [HuggingFace](https://huggingface.co/datasets/jeanlee/kmhas_korean_hate_speech) and checking meta information (published @COLING2022)


In [4]:
!pip uninstall datasets fsspec -y
!pip install datasets==3.2.0 fsspec[http]==2024.9.0


Found existing installation: fsspec 2024.10.0
Uninstalling fsspec-2024.10.0:
  Successfully uninstalled fsspec-2024.10.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_

In [5]:
!pip install transformers
!pip install datasets

In [6]:
from datasets import load_dataset

dataset = load_dataset("jeanlee/kmhas_korean_hate_speech")

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

kmhas_korean_hate_speech.py:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/5.24M [00:00<?, ?B/s]

default/validation/0000.parquet:   0%|          | 0.00/579k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78977 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8776 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/21939 [00:00<?, ? examples/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 78977
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 8776
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 21939
    })
})

In [8]:
dataset = load_dataset("jeanlee/kmhas_korean_hate_speech", split="test")

In [9]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 21939
})

In [10]:
dataset.features

{'text': Value(dtype='string', id=None),
 'label': Sequence(feature=ClassLabel(names=['origin', 'physical', 'politics', 'profanity', 'age', 'gender', 'race', 'religion', 'not_hate_speech'], id=None), length=-1, id=None)}

In [11]:
# meta information

print(dataset.info.description)
print(dataset.info.homepage)

The K-MHaS (Korean Multi-label Hate Speech) dataset contains 109k utterances from Korean online news comments labeled with 8 fine-grained hate speech classes or Not Hate Speech class.
The fine-grained hate speech classes are politics, origin, physical, age, gender, religion, race, and profanity and these categories are selected in order to reflect the social and historical context.

https://github.com/adlnlp/K-MHaS


In [12]:
print(dataset.info.citation)

@inproceedings{lee-etal-2022-k,
    title = "K-{MH}a{S}: A Multi-label Hate Speech Detection Dataset in {K}orean Online News Comment",
    author = "Lee, Jean  and
      Lim, Taejun  and
      Lee, Heejun  and
      Jo, Bogeun  and
      Kim, Yangsok  and
      Yoon, Heegeun  and
      Han, Soyeon Caren",
    booktitle = "Proceedings of the 29th International Conference on Computational Linguistics",
    month = oct,
    year = "2022",
    address = "Gyeongju, Republic of Korea",
    publisher = "International Committee on Computational Linguistics",
    url = "https://aclanthology.org/2022.coling-1.311",
    pages = "3530--3538",
    abstract = "Online hate speech detection has become an important issue due to the growth of online content, but resources in languages other than English are extremely limited. We introduce K-MHaS, a new multi-label dataset for hate speech detection that effectively handles Korean language patterns. The dataset consists of 109k utterances from news commen

## Data preparation

- Prepare data from train, validation, and test dataset
- Multi-label is converted to multi-label one hot encodding

      class_label:
        names:
          0: origin
          1: physical
          2: politics
          3: profanity
          4: age
          5: gender
          6: race
          7: religion
          8: not_hate_speech

In [13]:
!pip install transformers
!pip install datasets

In [14]:
from datasets import load_dataset

dataset = load_dataset("jeanlee/kmhas_korean_hate_speech")

In [15]:
!pip install keras_preprocessing


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.6 MB/s eta 0:00:00


In [16]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, hamming_loss
from sklearn.preprocessing import MultiLabelBinarizer

import pandas as pd
import numpy as np
import random
import time
import datetime
from tqdm import tqdm

import csv
import os



In [17]:
# load train, validation, and test dataset from HuggingFace

train = load_dataset("jeanlee/kmhas_korean_hate_speech", split="train")
validation = load_dataset("jeanlee/kmhas_korean_hate_speech", split="validation")
test = load_dataset("jeanlee/kmhas_korean_hate_speech", split="test")

In [18]:
# adding masking (able to remove this step depending on the model)

train_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', train['text']))
validation_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', validation['text']))
test_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', test['text']))

In [19]:
# convert multi-label to multi-label binary (one hot encoding)
# [8] -> [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]

from sklearn.preprocessing import MultiLabelBinarizer

enc = MultiLabelBinarizer()

def multi_label(example):
    enc_label = enc.fit_transform(example['label'])
    float_arr = np.vstack(enc_label[:]).astype(float)
    update_label = float_arr.tolist()
    return update_label

train_labels = multi_label(train)
validation_labels = multi_label(validation)
test_labels = multi_label(test)

In [20]:
test_sentences[:5]

['[CLS] 그만큼 길예르모가 잘했다고 보면되겠지 기대되네 셰이프 오브 워터 [SEP]',
 '[CLS] "1. 8넘의 문재앙" [SEP]',
 '[CLS] "문재인 정권의 내로남불은 타의 추종을 불허하네. 자한당 욕할거리도 없음." [SEP]',
 '[CLS] "짱개들 지나간 곳은 폐허된다 ㅋㅋ" [SEP]',
 '[CLS] 곱창은 자갈치~~~~~ [SEP]']

In [21]:
test_labels[:5]

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]

## Prep for Pytorch

In [22]:
# Tokenizing : bert-base-multilingual-cased

tokenizer = BertTokenizer.from_pretrained('klue/bert-base', do_lower_case=False)

In [23]:
MAX_LEN = 128

def data_to_tensor (sentences, labels):
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  attention_masks = []

  for seq in input_ids:
      seq_mask = [float(i > 0) for i in seq]
      attention_masks.append(seq_mask)

  tensor_inputs = torch.tensor(input_ids)
  tensor_labels = torch.tensor(labels)
  tensor_masks = torch.tensor(attention_masks)

  return tensor_inputs, tensor_labels, tensor_masks


In [24]:
train_inputs, train_labels, train_masks = data_to_tensor(train_sentences, train_labels)
validation_inputs, validation_labels, validation_masks = data_to_tensor(validation_sentences, validation_labels)
test_inputs, test_labels, test_masks = data_to_tensor(test_sentences, test_labels)

In [25]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [26]:
print('testset size:', len(test_labels))
print('trainset size:', len(train_labels))
print('validset size:', len(validation_labels))

testset size: 21939
trainset size: 78977
validset size: 8776


## Multi-BERT model

### GPU setting

In [27]:
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [28]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


### Model setting

In [29]:
num_labels = 9

model = BertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=num_labels, problem_type="multi_label_classification")
model.cuda()

model.safetensors:  24%|##3       | 105M/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [30]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                )

# change epochs for improving results (our paper : epochs = 4)
epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [31]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))  # hh:mm:ss

In [32]:
def multi_label_metrics(predictions, labels, threshold=0.5):

    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))

    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1

    # finally, compute metrics
    y_true = labels
    accuracy = accuracy_score(y_true, y_pred)
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0)
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0)
    f1_weighted_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=0)
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    hamming = hamming_loss(y_true, y_pred)

    # return as dictionary
    metrics = {'accuracy': accuracy,
               'f1_macro': f1_macro_average,
               'f1_micro': f1_micro_average,
               'f1_weighted': f1_weighted_average,
               'roc_auc': roc_auc,
               'hamming_loss': hamming}

    return metrics

### Model training

In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.zero_grad()
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_loss = 0

    model.train()

    for step, batch in tqdm(enumerate(train_dataloader)):
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # gradient clipping if it is over a threshold
        optimizer.step()
        scheduler.step()

        model.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)

    print("")
    print("  Average training loss: {0:.4f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...


500it [05:14,  1.55it/s]

  Batch   500  of  2,469.    Elapsed: 0:05:14.


1000it [10:36,  1.56it/s]

  Batch 1,000  of  2,469.    Elapsed: 0:10:37.


...

  Batch 1,500  of  2,469.    Elapsed: 0:16:07.


2000it [21:29,  1.55it/s]

  Batch 2,000  of  2,469.    Elapsed: 0:21:29.


2469it [26:31,  1.55it/s]


  Average training loss: 0.0499
  Training epcoh took: 0:26:31

Training complete!


In [ ]:
# ========================================
#               Validation
# ========================================

print("")
print("Running Validation...")

t0 = time.time()
model.eval()
accum_logits, accum_label_ids = [], []

for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    for b in logits:
        accum_logits.append(list(b))

    for b in label_ids:
        accum_label_ids.append(list(b))

accum_logits = np.array(accum_logits)
accum_label_ids = np.array(accum_label_ids)
results = multi_label_metrics(accum_logits, accum_label_ids)

print("Accuracy: {0:.4f}".format(results['accuracy']))
print("F1 (Macro) Score: {0:.4f}".format(results['f1_macro']))
print("F1 (Micro) Score: {0:.4f}".format(results['f1_micro']))
print("F1 (Weighted) Score: {0:.4f}".format(results['f1_weighted']))
print("ROC-AUC: {0:.4f}".format(results['roc_auc']))
print("Hamming Loss: {0:.4f}".format(results['hamming_loss']))
print("Validation took: {:}".format(format_time(time.time() - t0)))


Running Validation...
Accuracy: 0.8122
F1 (Macro) Score: 0.7599
F1 (Micro) Score: 0.8558
F1 (Weighted) Score: 0.8550
ROC-AUC: 0.9156
Hamming Loss: 0.0360
Validation took: 0:00:54


### Evaluation

In [ ]:
# model save

# torch.save(model.state_dict(), path+"BERT_model.pt")

In [34]:
# load the saved model

path = '/content/model/'
#torch.save(model.state_dict(), path+"BERT_multilabel_model.pt")
model.load_state_dict(torch.load(path+"BERT_multilabel_model.pt"))
model.to(device)

<ipython-input-34-d28ce5f3d4a8>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path+"BERT_multilabel_model.pt"))


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [1]:
%mkdir model

In [35]:
t0 = time.time()
model.eval()
accum_logits, accum_label_ids = [], []
for batch in validation_dataloader:
 batch = tuple(t.to(device) for t in batch)
 b_input_ids, b_input_mask, b_labels = batch
 with torch.no_grad():
  outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
 logits = outputs[0]
 logits = logits.detach().cpu().numpy()
 label_ids = b_labels.to('cpu').numpy()
 for b in logits:
  accum_logits.append(list(b))
 for b in label_ids:
  accum_label_ids.append(list(b))
accum_logits = np.array(accum_logits)
accum_label_ids = np.array(accum_label_ids)
results = multi_label_metrics(accum_logits, accum_label_ids)
print("Accuracy: {0:.4f}".format(results['accuracy']))
print("F1 (Macro) Score: {0:.4f}".format(results['f1_macro']))
print("F1 (Micro) Score: {0:.4f}".format(results['f1_micro']))
print("F1 (Weighted) Score: {0:.4f}".format(results['f1_weighted']))
print("ROC-AUC: {0:.4f}".format(results['roc_auc']))

Accuracy: 0.8122
F1 (Macro) Score: 0.7599
F1 (Micro) Score: 0.8558
F1 (Weighted) Score: 0.8550
ROC-AUC: 0.9156


In [36]:
t0 = time.time()
model.eval()
accum_logits, accum_label_ids = [], []

for step, batch in tqdm(enumerate(test_dataloader)):
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    for b in logits:
        accum_logits.append(list(b))

    for b in label_ids:
        accum_label_ids.append(list(b))

accum_logits = np.array(accum_logits)
accum_label_ids = np.array(accum_label_ids)
results = multi_label_metrics(accum_logits, accum_label_ids)

print("")
print("Accuracy: {0:.4f}".format(results['accuracy']))
print("F1 (Macro) Score: {0:.4f}".format(results['f1_macro']))
print("F1 (Micro) Score: {0:.4f}".format(results['f1_micro']))
print("F1 (Weighted) Score: {0:.4f}".format(results['f1_weighted']))
print("ROC-AUC: {0:.4f}".format(results['roc_auc']))
print("Hamming Loss: {0:.4f}".format(results['hamming_loss']))
print("Test took: {:}".format(format_time(time.time() - t0)))

accum_results = []
accum_results.append(list(results.values()))

100it [00:21,  4.64it/s]

  Batch   100  of    686.    Elapsed: 0:00:22.


200it [00:43,  4.79it/s]

  Batch   200  of    686.    Elapsed: 0:00:43.


300it [01:03,  4.79it/s]

...

  Batch   500  of    686.    Elapsed: 0:01:46.


600it [02:07,  4.70it/s]

  Batch   600  of    686.    Elapsed: 0:02:08.


686it [02:25,  4.70it/s]



Accuracy: 0.8073
F1 (Macro) Score: 0.7727
F1 (Micro) Score: 0.8549
F1 (Weighted) Score: 0.8545
ROC-AUC: 0.9149
Hamming Loss: 0.0366
Test took: 0:02:26


### Break down evaluation

In [37]:
for i in range(num_labels):
    ith_label_ids, ith_logits = [], []

    for j, labels in enumerate(accum_label_ids):
        if len(np.where(labels)[0]) == i+1:
            ith_label_ids.append(accum_label_ids[j].tolist())
            ith_logits.append(accum_logits[j].tolist())

    ith_label_ids = np.array(ith_label_ids)
    ith_logits = np.array(ith_logits)

    if len(ith_label_ids) == 0 and len(ith_logits) == 0:
        continue

    results = multi_label_metrics(ith_logits, ith_label_ids)
    accum_results.append(list(results.values()))

    print('# of labels:', i+1)
    print("Accuracy: {0:.4f}".format(results['accuracy']))
    print("F1 (Macro) Score: {0:.4f}".format(results['f1_macro']))
    print("F1 (Micro) Score: {0:.4f}".format(results['f1_micro']))
    print("F1 (Weighted) Score: {0:.4f}".format(results['f1_weighted']))
    print("ROC-AUC: {0:.4f}".format(results['roc_auc']))
    print("Hamming Loss: {0:.4f}".format(results['hamming_loss']))

    print('\n')

# of labels: 1
Accuracy: 0.8351
F1 (Macro) Score: 0.7588
F1 (Micro) Score: 0.8556
F1 (Weighted) Score: 0.8576
ROC-AUC: 0.9241
Hamming Loss: 0.0326


# of labels: 2
Accuracy: 0.6499
F1 (Macro) Score: 0.7102
F1 (Micro) Score: 0.8614
F1 (Weighted) Score: 0.8708
ROC-AUC: 0.8935
Hamming Loss: 0.0576


# of labels: 3
Accuracy: 0.3448
F1 (Macro) Score: 0.6232
F1 (Micro) Score: 0.8063
F1 (Weighted) Score: 0.8054
ROC-AUC: 0.8417
Hamming Loss: 0.1130


# of labels: 4
Accuracy: 0.2000
F1 (Macro) Score: 0.5363
F1 (Micro) Score: 0.7861
F1 (Weighted) Score: 0.7787
ROC-AUC: 0.8200
Hamming Loss: 0.1644




In [38]:
from transformers import pipeline
pipe = pipeline("text-classification", model=model.cuda(), tokenizer=tokenizer,
device=0, max_length=10,
 return_all_scores=True, function_to_apply='sigmoid')
result = pipe('틀니들은 왜 그렇게 민폐를 끼치냐?')
print(result)
label_dict = {'LABEL_0' : '출신차별', 'LABEL_1' : '외모차별', 'LABEL_2' : '정치성향차별', 'LABEL_3': '혐오욕설', 'LABEL_4': '연령차별', 'LABEL_5': '성차별', 'LABEL_6' : '인종차별', 'LABEL_7': '종교차별', 'LABEL_8': '해당사항없음'}
def prediction(text):
 result = pipe(text)
 return [label_dict[res['label']] for res in result[0] if res['score'] > 0.5]
prediction('틀니들은 왜 그렇게 민폐를 끼치냐?')

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[[{'label': 'LABEL_0', 'score': 0.006339981686323881}, {'label': 'LABEL_1', 'score': 0.007088158279657364}, {'label': 'LABEL_2', 'score': 0.007644087076187134}, {'label': 'LABEL_3', 'score': 0.005460667889565229}, {'label': 'LABEL_4', 'score': 0.9843930602073669}, {'label': 'LABEL_5', 'score': 0.010978417471051216}, {'label': 'LABEL_6', 'score': 0.0012231196742504835}, {'label': 'LABEL_7', 'score': 0.002903062617406249}, {'label': 'LABEL_8', 'score': 0.007692909799516201}]]


['연령차별']

In [39]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
from torch.nn.functional import sigmoid, softmax
from itertools import chain
import itertools
import pickle


label_names = ['출신차별', '외모차별', '정치성향차별', '혐오욕설', '연령차별', '성차별', '인종차별', '종교차별', '혐오아님']
threshold = 0.1 # 임계값 설정

def predict_labels(text, model, tokenizer, label_names, threshold=0.1):
    # 텍스트를 모델의 입력 형식으로 인코딩
    encoded_dict = tokenizer.encode_plus(
                        text,
                        add_special_tokens = True,
                        max_length = 10,
                        padding = 'max_length',
                        truncation=True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                        )

    input_ids = encoded_dict['input_ids'].to(device)
    attention_mask = encoded_dict['attention_mask'].to(device)

    # 모델을 사용하여 예측
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None, attention_mask=attention_mask)

    # 예측 결과에서 확률 추출
    logits = outputs.logits
    #print(logits)
    probs = sigmoid(logits)
    #print(probs)

    # CPU로 이동 후 numpy 배열로 변환
    probs = probs.detach().cpu().numpy()
    #print(probs)

    # 예측된 레이블 결정
    predicted_labels = [label_names[i] for i in range(len(label_names)) if probs[0][i] >= threshold]

    return predicted_labels

In [40]:
text = "늙은 천주교 신자들은 다 속물이다"
predicted_labels = predict_labels(text, model, tokenizer, label_names)
print(f"Text: {text} & Predicted labels: {predicted_labels}")

Text: 늙은 천주교 신자들은 다 속물이다 & Predicted labels: ['연령차별', '종교차별']


In [41]:
label_names = ['출신차별', '외모차별', '정치성향차별', '혐오욕설', '연령차별', '성차별', '인종차별', '종교차별', '혐오아님']
threshold = 0.1
text = "못생긴 경상도 여자들은 나가라"
predicted_labels = predict_labels(text, model, tokenizer, label_names)
print(f"Text: {text} -> Predicted labels: {predicted_labels}")

Text: 못생긴 경상도 여자들은 나가라 -> Predicted labels: ['출신차별', '외모차별', '성차별']


In [ ]:
# VSCode 쪽 코드 (receive_data.py)
outputs_2 = []

import pickle
with open("/./content/model/attention_scores.pkl", "rb") as f:
    outputs_2 = pickle.load(f)

print(outputs_2)

FileNotFoundError: [Errno 2] No such file or directory: '/./content/model/attention_scores.pkl'

In [42]:
result = pipe('문재앙')
print(result)

[[{'label': 'LABEL_0', 'score': 0.007721984758973122}, {'label': 'LABEL_1', 'score': 0.0032311684917658567}, {'label': 'LABEL_2', 'score': 0.985312283039093}, {'label': 'LABEL_3', 'score': 0.005993321072310209}, {'label': 'LABEL_4', 'score': 0.0053891874849796295}, {'label': 'LABEL_5', 'score': 0.002821417059749365}, {'label': 'LABEL_6', 'score': 0.0012292619794607162}, {'label': 'LABEL_7', 'score': 0.002137931529432535}, {'label': 'LABEL_8', 'score': 0.007046678103506565}]]


In [43]:
from transformers import pipeline

# GPU 사용 여부 확인
device = "cpu"

# 모델을 CPU로 이동
model = model.to(device)

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=device, max_length=10, return_all_scores=True)

result = pipe('깜둥이들은 왜 그렇게 민폐를 끼치냐?')
print(result)

Device set to use cpu


[[{'label': 'LABEL_0', 'score': 0.14709047973155975}, {'label': 'LABEL_1', 'score': 0.26493778824806213}, {'label': 'LABEL_2', 'score': 0.026825927197933197}, {'label': 'LABEL_3', 'score': 0.03060534968972206}, {'label': 'LABEL_4', 'score': 0.08758819103240967}, {'label': 'LABEL_5', 'score': 0.09902970492839813}, {'label': 'LABEL_6', 'score': 0.8641231060028076}, {'label': 'LABEL_7', 'score': 0.14547504484653473}, {'label': 'LABEL_8', 'score': 0.11148741096258163}]]


In [44]:
text = "ㅅ발 천주교도들은 너무 말이 많아"
predicted_labels = predict_labels(text, model, tokenizer, label_names)
print(f"Text: {text} & Predicted labels: {predicted_labels}")

Text: ㅅ발 천주교도들은 너무 말이 많아 & Predicted labels: ['혐오욕설', '종교차별']


In [45]:
# 파일을 열고 읽은 다음 내용을 출력하는 코드
file_path = '/home/jyhan/HW-output-files/example.txt'  # 읽고자 하는 파일의 경로

try:
    with open(file_path, 'r') as file:  # 파일을 읽기 모드로 열기
        content = file.read()  # 파일의 내용을 전체 읽기
    print(content)  # 읽은 내용 출력
except FileNotFoundError:
    print(f"The file {file_path} does not exist.")  # 파일이 존재하지 않을 경우 에러 메시지 출력

The file /home/jyhan/HW-output-files/example.txt does not exist.


In [46]:
import pickle

def pickle_to_text(pickle_file_path, text_file_path):
    # 피클 파일 불러오기
    with open(pickle_file_path, "rb") as f:
        data = pickle.load(f)

    # 텍스트 파일로 저장하기
    with open(text_file_path, "w") as f:
        for sublist in data:
            # 각 서브리스트를 반복하고 숫자를 문자열로 변환하여 저장
            for number in sublist:
                f.write(f"{number} ")
            f.write("\n\n\n")  # 각 서브리스트 끝에 줄바꿈 추가

In [47]:
import torch
import torch.nn.functional as F

def tanh_new(x) :
    result = (F.tanh(x) + 1) /2
    return result

In [80]:
def encode_number(num):
    """숫자를 7비트 2의 보수 형식으로 인코딩하는 함수, 음수 소수 부분을 고려"""
    num = float(num)  # 입력된 숫자가 문자열이라면, 실수형으로 변환

    # Extract integer and fractional parts
    int_num = int(num)
    frac_num = abs(num - int_num)

    # Round integer part towards zero if num is negative and has a fractional part
    if num < 0:
        if frac_num > 0:
            int_num = int_num - 1  # Round integer part one more negative
            frac_num = 1 - frac_num  # Subtract fractional part from 1 to make it positive

    # Clamp the values to fit within the 7-bit range
    if int_num < -64:
        int_num = -64
    elif int_num > 63:
        int_num = 63

    # Apply 2's complement if the number is negative
    if int_num < 0:
        int_num = (1 << 7) + int_num  # 1 << 7 is 128, representing the range of 7-bit integers

    # Format the number into 7-bit binary
    int_part_bin = format(int_num & 0b1111111, '07b')  # Only the last 7 bits are used

    # Convert the fractional part to binary (13 bits)
    decimal_part_bin = format(int(frac_num * (1 << 13)), '013b')

    # Combine all parts
    encoded = int_part_bin + '_' + decimal_part_bin
    return encoded

In [48]:
def encode_numbers_input(numbers):
    """리스트의 숫자들을 인코딩하는 함수"""
    return [encode_number(num) for num in numbers]

In [49]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
from torch.nn.functional import sigmoid, softmax
from itertools import chain
import itertools

In [50]:
def encode_number_output(num):
    """숫자를 3비트 정수와 13비트 소수 형식으로 인코딩하는 함수, 값은 0부터 1 사이의 양수"""
    num = float(num)  # 입력된 숫자가 문자열이라면, 실수형으로 변환

    # Clamp the values to fit within the 0 to 1 range
    if num < 0:
        num = 0
    elif num > 1:
        num = 1

    # Extract integer and fractional parts
    int_num = int(num)
    frac_num = num - int_num

    # Convert the fractional part to binary (13 bits)
    decimal_part_bin = format(int(frac_num * (1 << 13)), '013b')

    # Combine all parts
    encoded = '000' + '_' + decimal_part_bin
    return encoded

In [51]:
def encode_numbers_output(numbers):
    """리스트의 숫자들을 인코딩하는 함수"""
    return [encode_number_output(num) for num in numbers]

In [55]:
probs = [-3.0061, -4.5561, -3.7781, -4.6988, -1.8251, -4.0483, -4.8075,  4.6085, -5.1693]
encoded_numbers = encode_numbers_input(probs)
print(encoded_numbers)

probs = [0.0472, 0.0104, 0.0224, 0.0090, 0.1388, 0.0172, 0.0081, 0.9901, 0.0057]
encoded_numbers = encode_numbers_output(probs)
print(encoded_numbers)

NameError: name 'encode_number' is not defined

In [53]:
probs = [0.0301, 0.0112, 0.0040, 0.7632, 0.0040, 0.0072, 0.0068, 0.8885, 0.0117]
encoded_numbers = encode_numbers_output(probs)
print(encoded_numbers)

['000_0000011110110', '000_0000001011011', '000_0000000100000', '000_1100001101100', '000_0000000100000', '000_0000000111010', '000_0000000110111', '000_1110001101110', '000_0000001011111']


In [54]:
def decode_number(encoded):
    """7비트 2의 보수 형식으로 인코딩된 값을 float으로 디코딩하는 함수"""
    int_part_bin, decimal_part_bin = encoded.split('.')

    # Integer part processing
    if int(int_part_bin, 2) & (1 << 6):  # Negative number (2의 보수)
        int_num = int(int_part_bin, 2) - (1 << 7)
    else:  # Positive number
        int_num = int(int_part_bin, 2)

    # Fractional part processing
    frac_num = int(decimal_part_bin, 2) / (1 << 13)

    # Adjust for negative numbers
    if int_num < 0:
        if frac_num > 0:
            int_num += 1
            frac_num = 1 - frac_num
        result = int_num - frac_num
    else:
        result = int_num + frac_num

    return result

# Test the function with provided values
encoded_values = [
    "0000000.0101010000101",
    "0000000.0010111100000",
    "0000000.0010110110000",
    "0000000.0010001111011",
    "0000000.0010001101110",
    "0000000.0001110010000",
    "0000000.0001100000110",
    "0000000.0000000000010",
    "1111111.1101100010111",
    "1111111.1101011100011"
]

decoded_values = [decode_number(value) for value in encoded_values]
decoded_values

[0.3287353515625,
 0.18359375,
 0.177734375,
 0.1400146484375,
 0.138427734375,
 0.111328125,
 0.094482421875,
 0.000244140625,
 -0.1534423828125,
 -0.1597900390625]

In [56]:
def decode_number(encoded):
    """7비트 2의 보수 형식으로 인코딩된 값을 float으로 디코딩하는 함수"""
    # 입력 문자열을 반으로 나누어 정수 부분과 소수 부분으로 분리
    mid_index = len(encoded) // 2
    int_part_bin = encoded[:mid_index]
    decimal_part_bin = encoded[mid_index:]

    # 정수 부분 처리
    if int(int_part_bin, 2) & (1 << 6):  # 음수인 경우 (2의 보수)
        int_num = int(int_part_bin, 2) - (1 << 7)
    else:  # 양수인 경우
        int_num = int(int_part_bin, 2)

    # 소수 부분 처리
    frac_num = int(decimal_part_bin, 2) / (1 << 13)

    # 음수 조정
    if int_num < 0:
        if frac_num > 0:
            int_num += 1
            frac_num = 1 - frac_num
        result = int_num - frac_num
    else:
        result = int_num + frac_num

    return result

# 제공된 값으로 함수를 테스트합니다
encoded_values = [
    "00000000101010000101",
    "00000000010111100000",
    "00000000010110110000",
    "00000000010001111011",
    "00000000010001101110",
    "00000000001110010000",
    "00000000001100000110",
    "00000000000000000010",
    "11111111101100010111",
    "11111111101011100011"
]

decoded_values = [decode_number(value) for value in encoded_values]
decoded_values

[2.0787353515625,
 1.05859375,
 1.052734375,
 1.0150146484375,
 1.013427734375,
 0.111328125,
 0.094482421875,
 0.000244140625,
 894.0965576171875,
 894.0902099609375]

In [57]:
def decode_int3_float13(encoded):
    """3비트 정수 + 13비트 소수 형식으로 인코딩된 값을 float으로 디코딩하는 함수"""
    int_part_bin = encoded[:3]
    decimal_part_bin = encoded[3:]

    # 정수 부분 처리 (3비트)
    if int(int_part_bin, 2) & (1 << 2):  # 음수인 경우 (2의 보수)
        int_num = int(int_part_bin, 2) - (1 << 3)
    else:  # 양수인 경우
        int_num = int(int_part_bin, 2)

    # 소수 부분 처리 (13비트)
    frac_num = int(decimal_part_bin, 2) / (1 << 13)

    # 음수 조정
    if int_num < 0:
        if frac_num > 0:
            int_num += 1
            frac_num = 1 - frac_num
        result = int_num - frac_num
    else:
        result = int_num + frac_num

    return result

# 제공된 값으로 함수를 테스트합니다
encoded_values = [
    "0001000101001100",
    "0000011001010100",
    "0000000111011100",
    "0000000111010100",
    "0000000110110100",
    "0000000100000100",
    "0000000011010100",
    "0000000001100100",
    "0000000001010100",
    "0000000001000100"
]

decoded_values = [decode_int3_float13(value) for value in encoded_values]

# 결과 출력
for encoded, decoded in zip(encoded_values, decoded_values):
    print(f"Encoded: {encoded} -> Decoded: {decoded}")

Encoded: 0001000101001100 -> Decoded: 0.54052734375
Encoded: 0000011001010100 -> Decoded: 0.19775390625
Encoded: 0000000111011100 -> Decoded: 0.05810546875
Encoded: 0000000111010100 -> Decoded: 0.05712890625
Encoded: 0000000110110100 -> Decoded: 0.05322265625
Encoded: 0000000100000100 -> Decoded: 0.03173828125
Encoded: 0000000011010100 -> Decoded: 0.02587890625
Encoded: 0000000001100100 -> Decoded: 0.01220703125
Encoded: 0000000001010100 -> Decoded: 0.01025390625
Encoded: 0000000001000100 -> Decoded: 0.00830078125


In [58]:
def format_hex_input(hex_input):
    # 공백을 기준으로 입력값을 분할하여 리스트로 변환
    hex_list = hex_input.split()

    # 리스트를 문자열 형식으로 변환
    formatted_hex_list = "[" + ", ".join(f'"{hex_value}"' for hex_value in hex_list) + "]"

    return formatted_hex_list

# 입력값
hex_input = "06dc 05c4 0494 03c4 039c 02e4 017c 0144 00e4 00ac"
formatted_hex_list = format_hex_input(hex_input)

print(formatted_hex_list)

["06dc", "05c4", "0494", "03c4", "039c", "02e4", "017c", "0144", "00e4", "00ac"]


In [59]:
def hexa_to_binary(hex_strings):
    binary_strings = [bin(int(hex_string, 16))[2:].zfill(16) for hex_string in hex_strings]
    return binary_strings

def decode_int3_float13(encoded):
    """3비트 정수 + 13비트 소수 형식으로 인코딩된 값을 float으로 디코딩하는 함수"""
    int_part_bin = encoded[:3]
    decimal_part_bin = encoded[3:]

    # 정수 부분 처리 (3비트)
    int_num = int(int_part_bin, 2)

    # 소수 부분 처리 (13비트)
    frac_num = int(decimal_part_bin, 2) / (1 << 13)

    # 결과 계산
    result = int_num + frac_num

    return result

def hexa_to_float(hex_strings):
    binary_strings = hexa_to_binary(hex_strings)
    float_values = [decode_int3_float13(binary_string) for binary_string in binary_strings]
    return float_values

# 새로운 입력값
hex_strings = ["06dc", "05c4", "0494", "03c4", "039c", "02e4", "017c", "0144", "00e4", "00ac"]

float_values = hexa_to_float(hex_strings)

# 변환된 결과 출력
for hex_string, float_value in zip(hex_strings, float_values):
    print(f"{float_value}")

0.21435546875
0.18017578125
0.14306640625
0.11767578125
0.11279296875
0.09033203125
0.04638671875
0.03955078125
0.02783203125
0.02099609375


In [60]:
def format_hex_input(hex_input):
    # 공백을 기준으로 입력값을 분할하여 리스트로 변환
    hex_list = hex_input.split()
    return hex_list

def hexa_to_binary(hex_strings):
    binary_strings = [bin(int(hex_string, 16))[2:].zfill(16) for hex_string in hex_strings]
    return binary_strings

def decode_int3_float13(encoded):
    """3비트 정수 + 13비트 소수 형식으로 인코딩된 값을 float으로 디코딩하는 함수"""
    int_part_bin = encoded[:3]
    decimal_part_bin = encoded[3:]

    # 정수 부분 처리 (3비트)
    int_num = int(int_part_bin, 2)

    # 소수 부분 처리 (13비트)
    frac_num = int(decimal_part_bin, 2) / (1 << 13)

    # 결과 계산
    result = int_num + frac_num

    return result

def hexa_to_float(hex_strings):
    binary_strings = hexa_to_binary(hex_strings)
    float_values = [decode_int3_float13(binary_string) for binary_string in binary_strings]
    return float_values

def convert_hex_input_to_float(hex_input):
    hex_strings = format_hex_input(hex_input)
    float_values = hexa_to_float(hex_strings)
    return float_values

def chunk_floats(float_values, chunk_size=10):
    # float 값을 chunk_size 크기로 분할하고 각 chunk 사이에 두 개의 엔터를 추가
    chunked_result = ""
    for i in range(0, len(float_values), chunk_size):
        chunk = float_values[i:i + chunk_size]
        chunked_result += "\n".join(map(str, chunk)) + "\n\n"
    return chunked_result.strip()

# 입력값
hex_input = """
1404
026c
0234
0214
0184
017c
015c
001c
001c
0004
"""

float_values = convert_hex_input_to_float(hex_input)

# 변환된 결과 출력
chunked_result = chunk_floats(float_values)

print(chunked_result)

0.62548828125
0.07568359375
0.06884765625
0.06494140625
0.04736328125
0.04638671875
0.04248046875
0.00341796875
0.00341796875
0.00048828125


In [61]:
def decode_int3_float13(encoded):
    """3비트 정수 + 13비트 소수 형식으로 인코딩된 값을 float으로 디코딩하는 함수"""
    int_part_bin = encoded[:3]
    decimal_part_bin = encoded[3:]

    # 정수 부분 처리 (3비트)
    int_num = int(int_part_bin, 2)

    # 소수 부분 처리 (13비트)
    frac_num = int(decimal_part_bin, 2) / (1 << 13)

    # 결과 계산
    result = int_num + frac_num

    return result

# 제공된 값으로 함수를 테스트합니다
encoded_values = [
    "0000110101000100",
    "0000100000100100",
    "0000001101111100",
    "0000001011111100",
    "0000000111000100",
    "0000000100000100",
    "0000000001110100",
    "0000000001000100",
    "0000000000111100",
    "0000000000110100"
]

decoded_values = [decode_int3_float13(value) for value in encoded_values]

# 결과 출력
for decoded in decoded_values:
    print(decoded)


0.41455078125
0.25439453125
0.10888671875
0.09326171875
0.05517578125
0.03173828125
0.01416015625
0.00830078125
0.00732421875
0.00634765625


In [62]:
# 새로운 입력값
new_hex_strings = ["0d44", "0824", "037c", "02fc", "01c4", "0104", "0074", "0044", "003c", "0034"]
new_binary_strings = hexa_to_binary(new_hex_strings)

# 변환된 결과 출력
for hex_string, binary_string in zip(new_hex_strings, new_binary_strings):
    print(f"{binary_string}")


0000110101000100
0000100000100100
0000001101111100
0000001011111100
0000000111000100
0000000100000100
0000000001110100
0000000001000100
0000000000111100
0000000000110100


In [63]:
def hexa_to_binary(hex_strings):
    binary_strings = [bin(int(hex_string, 16))[2:].zfill(16) for hex_string in hex_strings]
    return binary_strings

# 새로운 입력값
hex_strings = ["069c", "046c", "045c", "0434", "02fc", "02a4", "026c", "01d4", "00e4"]
binary_strings = hexa_to_binary(hex_strings)

# 변환된 결과 출력
for hex_string, binary_string in zip(hex_strings, binary_strings):
    print(f"{binary_string}")

0000011010011100
0000010001101100
0000010001011100
0000010000110100
0000001011111100
0000001010100100
0000001001101100
0000000111010100
0000000011100100


In [64]:
# 새로운 입력값
hex_strings = ["045c", "0404", "03b4", "039c", "0374", "034c", "031c", "027c", "022c", "01bc"]
binary_strings = hexa_to_binary(hex_strings)

# 변환된 결과 출력
for hex_string, binary_string in zip(hex_strings, binary_strings):
    print(f"{binary_string}")


0000010001011100
0000010000000100
0000001110110100
0000001110011100
0000001101110100
0000001101001100
0000001100011100
0000001001111100
0000001000101100
0000000110111100


In [65]:
def calculate_error_rates(actual_values, predicted_values):
    """
    여러 개의 실제 값과 예측 값에 대한 오차율을 계산하는 함수.

    매개변수:
    actual_values (list of float): 실제 값 리스트
    predicted_values (list of float): 예측 값 리스트

    반환값:
    list of float: 각 데이터에 대한 오차율 (백분율) 리스트
    """
    if len(actual_values) != len(predicted_values):
        raise ValueError("실제 값 리스트와 예측 값 리스트의 길이는 같아야 합니다.")

    error_rates = []
    for actual, predicted in zip(actual_values, predicted_values):
        if actual == 0:
            error_rate = 0
        else:
            error = actual - predicted
            error_rate = abs((error / actual) * 100)
        error_rates.append(error_rate)

    return error_rates

def calculate_average_error_rate(actual_values, predicted_values):
    """
    여러 개의 실제 값과 예측 값에 대한 평균 오차율을 계산하는 함수.

    매개변수:
    actual_values (list of float): 실제 값 리스트
    predicted_values (list of float): 예측 값 리스트

    반환값:
    float: 평균 오차율 (백분율)
    """
    error_rates = calculate_error_rates(actual_values, predicted_values)
    average_error_rate = sum(error_rates) / len(error_rates)
    return average_error_rate

In [66]:
# 예제 사용
actual_values = [3.447215, 2.4453104, 1.2259121, 1.2043934, 1.1346276, 0.6330924, 0.4353165, -0.2769869, -0.47311664, -0.6537097]
predicted_values = [3.447215, 2.4453104, 1.2259121, 1.2043934, 1.1346276, 0.6330924, 0.4353165, -0.2769869, -0.47311664, -0.6537097]
average_error_rate = calculate_average_error_rate(actual_values, predicted_values)

print(f"최종 오차율: {average_error_rate}%")

최종 오차율: 0.0%


In [67]:
import time
import numpy as np

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

# 10개의 값을 생성
values = np.random.rand(10)

# 실행 시간 측정
start_time = time.time()
softmax_values = softmax(values)
end_time = time.time()

print(f"Softmax output: {softmax_values}")
print(f"Execution time: {end_time - start_time} seconds")

Softmax output: [0.09222514 0.07933714 0.13408144 0.07743605 0.13117615 0.1283651
 0.09086202 0.08387534 0.0808663  0.10177532]
Execution time: 0.0003597736358642578 seconds


In [68]:
import timeit
import numpy as np

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

# 10개의 값을 생성
values = np.random.rand(10)

# 실행 시간 측정 (timeit 모듈 사용)
execution_time = timeit.timeit(lambda: softmax(values), number=100000)
print(f"Average execution time over 1000 runs: {execution_time / 100000} seconds")


Average execution time over 1000 runs: 7.871906190000572e-06 seconds


In [69]:
import time

def my_function():
    # 실행할 코드
    time.sleep(2)  # 예시로 2초 지연

start_time = time.time()
my_function()
end_time = time.time()

print(f"Execution time: {end_time - start_time} seconds")

Execution time: 2.000333786010742 seconds


In [70]:
import timeit

def my_function():
    # 실행할 코드
    time.sleep(2)  # 예시로 2초 지연

execution_time = timeit.timeit(my_function, number=1)
print(f"Execution time: {execution_time} seconds")


Execution time: 2.000072781000199 seconds


In [71]:
import time

def measure_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Execution time: {end_time - start_time} seconds")
        return result
    return wrapper

@measure_time
def my_function():
    # 실행할 코드
    time.sleep(2)  # 예시로 2초 지연

my_function()

Execution time: 2.0000922679901123 seconds


In [72]:

import socket

# HW -> SW connect
# This should be fixed later when connecting with hardware.
def receive_from_hardware(host: str, port: int, buffer_size: int = 1024) -> bytes:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((host, port))
        attention_probs_hw = s.recv(buffer_size)
    return attention_probs_hw

# HW 2진수 -> SW Decode
def decode_values(encoded_values_list: list) -> list:
    encoded_values = tuple(encoded_values_list)
    decoded_values = []

    for encoded_value in encoded_values:
        special_value_bit = encoded_value[0]
        if special_value_bit == '1':
            decoded_values.append(0.0)
        else:
            sign_bit = encoded_value[1]
            sign = -1 if sign_bit == '1' else 1

            integer_part = int(encoded_value[2:5], 2)
            fractional_part = int(encoded_value[5:], 2) / (1 << 13)

            decoded_value = sign * (integer_part + fractional_part)
            decoded_values.append(round(decoded_value, 7))

    return decoded_values

# 1D -> 4D 변환 코드
def convert_to_4d(input_list):
    # Step 1: Divide the list into sublists of 10 elements each
    sublists = [input_list[i:i + 10] for i in range(0, len(input_list), 10)]

    # Step 2: Group every 10 sublists into a larger list to form a [12, 10, 10] shape
    grouped_sublists = [sublists[i:i + 10] for i in range(0, len(sublists), 10)]

    # Step 3: Convert the final list to a tensor and add an extra dimension to form [1, 12, 10, 10]
    attention_probs_4d = torch.tensor(grouped_sublists).unsqueeze(0)
    return attention_probs_4d

In [111]:
import pickle
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# attention_scores 저장 함수 정의
def save_attention_scores(attention_scores, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(attention_scores, f)


# 파일 경로 지정
file_path = './model/attention_scores.pkl'

# 함수 호출
save_attention_scores(layer_1_attention, file_path)

print(f"Attention scores가 {file_path}에 저장되었습니다!")


Attention scores가 ./model/attention_scores.pkl에 저장되었습니다!


In [85]:
import pickle
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# attention_scores 저장 함수 정의
def save_attention_scores(attention_scores, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(attention_scores, f)


# 파일 경로 지정
file_path = './model/attention_scores.pkl'

# 함수 호출
save_attention_scores(layer_1_raw_attention, file_path)

print(f"Attention scores가 {file_path}에 저장되었습니다!")

Attention scores가 ./model/attention_scores.pkl에 저장되었습니다!


In [87]:
pickle_file_path = "/./content/model/attention_scores.pkl"
text_file_path = "/./content/model/attention_scores_2.txt"
pickle_to_text(pickle_file_path, text_file_path)

In [ ]:
import pickle
import math
attention_scores = layer_1_attention
attention_scores = attention_scores / math.sqrt(self.attention_head_size)
self.all_head_size = self.num_attention_heads * self.attention_head_size
 #Pickle 파일 저장
if attention_mask is not None:
 # Apply the attention mask is (precomputed for all layers in BertModel
 attention_scores = attention_scores + attention_mask
 # attention_scores를 피클 파일로 저장합니다.
attention_scores_list = attention_scores.tolist()  # torch.Tensor를 Python 리스트로 변환
with open("/./content/model/attention_scores.pkl", "wb") as f:
 pickle.dump(attention_scores_list, f)
 #저장된 Pickle 파일 불러와서 plot
with open("/./content/model/attention_scores.pkl", "rb") as f:
 outputs = pickle.load(f)
out_chain3 = list(itertools.chain(*outputs))
out_chain2 = list(itertools.chain(*out_chain3))
out_chain1 = list(itertools.chain(*out_chain2))
print(out_chain1)
max_value = torch.max(attention_scores)
min_value = torch.min(attention_scores)
print("Max value in attention_scores:", max_value.item())

NameError: name 'self' is not defined

In [73]:
from typing import Optional, Tuple
def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        encoder_hidden_states: Optional[torch.FloatTensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        past_key_value: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
        output_attentions: Optional[bool] = False,
    ) -> Tuple[torch.Tensor]:
        mixed_query_layer = self.query(hidden_states)

        # If this is instantiated as a cross-attention module, the keys
        # and values come from an encoder; the attention mask needs to be
        # such that the encoder's padding tokens are not attended to.
        is_cross_attention = encoder_hidden_states is not None

        if is_cross_attention and past_key_value is not None:
            # reuse k,v, cross_attentions
            key_layer = past_key_value[0]
            value_layer = past_key_value[1]
            attention_mask = encoder_attention_mask
        elif is_cross_attention:
            key_layer = self.transpose_for_scores(self.key(encoder_hidden_states))
            value_layer = self.transpose_for_scores(self.value(encoder_hidden_states))
            attention_mask = encoder_attention_mask
        elif past_key_value is not None:
            key_layer = self.transpose_for_scores(self.key(hidden_states))
            value_layer = self.transpose_for_scores(self.value(hidden_states))
            key_layer = torch.cat([past_key_value[0], key_layer], dim=2)
            value_layer = torch.cat([past_key_value[1], value_layer], dim=2)
        else:
            key_layer = self.transpose_for_scores(self.key(hidden_states))
            value_layer = self.transpose_for_scores(self.value(hidden_states))

        query_layer = self.transpose_for_scores(mixed_query_layer)

        use_cache = past_key_value is not None
        if self.is_decoder:
            # if cross_attention save Tuple(torch.Tensor, torch.Tensor) of all cross attention key/value_states.
            # Further calls to cross_attention layer can then reuse all cross-attention
            # key/value_states (first "if" case)
            # if uni-directional self-attention (decoder) save Tuple(torch.Tensor, torch.Tensor) of
            # all previous decoder key/value_states. Further calls to uni-directional self-attention
            # can concat previous decoder key/value_states to current projected key/value_states (third "elif" case)
            # if encoder bi-directional self-attention `past_key_value` is always `None`
            past_key_value = (key_layer, value_layer)

        # Take the dot product between "query" and "key" to get the raw attention scores.
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))

        if self.position_embedding_type == "relative_key" or self.position_embedding_type == "relative_key_query":
            query_length, key_length = query_layer.shape[2], key_layer.shape[2]
            if use_cache:
                position_ids_l = torch.tensor(key_length - 1, dtype=torch.long, device=hidden_states.device).view(
                    -1, 1
                )
            else:
                position_ids_l = torch.arange(query_length, dtype=torch.long, device=hidden_states.device).view(-1, 1)
            position_ids_r = torch.arange(key_length, dtype=torch.long, device=hidden_states.device).view(1, -1)
            distance = position_ids_l - position_ids_r

            positional_embedding = self.distance_embedding(distance + self.max_position_embeddings - 1)
            positional_embedding = positional_embedding.to(dtype=query_layer.dtype)  # fp16 compatibility

            if self.position_embedding_type == "relative_key":
                relative_position_scores = torch.einsum("bhld,lrd->bhlr", query_layer, positional_embedding)
                attention_scores = attention_scores + relative_position_scores
            elif self.position_embedding_type == "relative_key_query":
                relative_position_scores_query = torch.einsum("bhld,lrd->bhlr", query_layer, positional_embedding)
                relative_position_scores_key = torch.einsum("bhrd,lrd->bhlr", key_layer, positional_embedding)
                attention_scores = attention_scores + relative_position_scores_query + relative_position_scores_key

        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        if attention_mask is not None:
            # Apply the attention mask is (precomputed for all layers in BertModel forward() function)
            attention_scores = attention_scores + attention_mask

        # Normalize the attention scores to probabilities.
        attention_probs = nn.functional.softmax(attention_scores, dim=-1)

        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(attention_probs)

        # Mask heads if we want to
        if head_mask is not None:
            attention_probs = attention_probs * head_mask

        context_layer = torch.matmul(attention_probs, value_layer)

        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(new_context_layer_shape)

        outputs = (context_layer, attention_probs) if output_attentions else (context_layer,)

        if self.is_decoder:
            outputs = outputs + (past_key_value,)
        return outputs

In [74]:
from transformers import BertTokenizer, BertModel
import torch

class BertWithRawAttention(BertModel):
    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, inputs_embeds=None, output_attentions=True, output_hidden_states=False, return_dict=True):
        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=True,  # Attention 값 활성화
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        # Attention Score (Softmax 이후 값)
        attention_probs = outputs.attentions  # (batch, num_heads, seq_length, seq_length)

        # Softmax 이전 값 = Scaled Dot-Product Attention Score (QK^T / sqrt(d_k))
        raw_attentions = []
        for layer in self.encoder.layer:
            Q = layer.attention.self.query(outputs.last_hidden_state)
            K = layer.attention.self.key(outputs.last_hidden_state)
            d_k = Q.shape[-1] ** 0.5  # sqrt(d_k)
            raw_attention = torch.matmul(Q, K.transpose(-2, -1)) / d_k  # QK^T / sqrt(d_k)
            raw_attentions.append(raw_attention)

        return raw_attentions, attention_probs  # Softmax 이전 값과 이후 값 반환

# 모델 로드
model_name = "klue/bert-base"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertWithRawAttention.from_pretrained(model_name)

# 입력 문장
text = "ㅅ발 천주교도들은 너무 말이 많아"
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=10)

# 모델 실행
model.eval()
with torch.no_grad():
    raw_attentions, softmax_attentions = model(**inputs)

# 첫 번째 레이어의 Softmax 이전 Attention Score 출력
layer_1_raw_attention = raw_attentions[0].numpy()  # NumPy 변환
print(f"Layer 1 Softmax 이전 Attention Score (Shape: {layer_1_raw_attention.shape}):")
print(layer_1_raw_attention)


BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


Layer 1 Softmax 이전 Attention Score (Shape: (1, 10, 10)):
[[[-1.2585267e+00  2.6621759e-01 -8.7831020e-01 -5.4661173e-01
   -2.8880724e-01  1.1142263e+00 -7.5586647e-01 -1.0404842e+00
   -6.2354100e-01 -4.8658773e-01]
  [-2.5612357e-01 -1.7154795e-01 -8.6090702e-01 -7.6754767e-01
   -9.9591351e-01 -1.0064337e+00 -4.0744746e-01 -1.7507937e+00
    1.7926955e-01  1.0923295e+00]
  [-2.9597259e-01 -5.9717542e-01 -5.0122070e-01 -7.3208112e-01
   -9.1411477e-01 -4.8521334e-01 -9.0301853e-01 -1.1684108e+00
   -1.5857342e-01  1.2634809e+00]
  [-1.3794887e-01 -2.0422029e-01  4.0585518e-01 -1.5016638e+00
    5.4751128e-02  1.4497165e-01 -1.1364064e+00 -2.2216363e+00
   -2.4729875e-01 -1.3346322e+00]
  [ 1.0132506e+00  1.6417295e+00  6.2383556e-01  6.2764668e-01
   -6.2279487e-01  9.1525161e-01  3.9802289e-01 -1.1625541e+00
   -6.0396677e-01 -1.1437993e+00]
  [ 1.1180567e+00  9.2262423e-01  1.6634616e+00  1.2863833e+00
    3.0124974e-01 -3.3511552e-01  4.4247529e-01 -2.0134099e+00
    1.5760978e-01

In [75]:
from transformers import BertTokenizer, BertModel
import torch

class BertWithRawAttention(BertModel):
    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, inputs_embeds=None, output_attentions=True, output_hidden_states=False, return_dict=True):
        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=True,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        num_heads = self.config.num_attention_heads  # KLUE-BERT의 Head 수 (12)
        hidden_dim = self.config.hidden_size  # BERT의 Hidden Dimension (768)
        head_dim = hidden_dim // num_heads  # 각 Head의 크기
        d_k = head_dim ** 0.5  # sqrt(d_k)

        raw_attentions = []
        for layer in self.encoder.layer:
            Q = layer.attention.self.query(outputs.last_hidden_state)  # (1, 10, 768)
            K = layer.attention.self.key(outputs.last_hidden_state)  # (1, 10, 768)

            # 🚀 **멀티헤드 형태로 변환 (12개 헤드 고려)**
            Q = Q.view(1, 10, num_heads, head_dim).transpose(1, 2)  # (1, 12, 10, head_dim)
            K = K.view(1, 10, num_heads, head_dim).transpose(1, 2)  # (1, 12, 10, head_dim)

            # Softmax 이전 Attention Score (1, 12, 10, 10)
            raw_attention = torch.matmul(Q, K.transpose(-2, -1)) / d_k
            raw_attentions.append(raw_attention)

        return raw_attentions, outputs.attentions  # Softmax 이전 & 이후 값 반환

# 모델 로드
model_name = "klue/bert-base"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertWithRawAttention.from_pretrained(model_name)

# 입력 문장
text = "ㅅ발 천주교도들은 너무 말이 많아"
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=10)

# 모델 실행
model.eval()
with torch.no_grad():
    raw_attentions, softmax_attentions = model(**inputs)

#  Softmax 이전 값의 shape이 (1, 12, 10, 10)
layer_1_raw_attention = raw_attentions[0].numpy()
print(f"Layer 1 Softmax 이전 Attention Score (Shape: {layer_1_raw_attention.shape}):")
print(layer_1_raw_attention)


Layer 1 Softmax 이전 Attention Score (Shape: (1, 12, 10, 10)):
[[[[-0.16330126  0.74665904  0.5046175  ...  0.1053734  -0.3517992
    -0.82628703]
   [-0.85504246 -0.8495395   0.27420458 ... -0.7120759  -0.6404201
    -0.7213575 ]
   [-0.33850312 -0.49887422 -1.0850958  ... -0.41751945 -0.6837665
     0.12196846]
   ...
   [ 0.04038128 -1.5239407  -0.4461439  ... -0.8514781  -0.62913764
    -1.6456957 ]
   [-0.48097274 -1.1991742  -0.16329284 ...  0.6281986  -0.76396775
    -0.7268485 ]
   [-0.87766147 -1.1943011  -0.7693224  ... -1.3699665  -1.3390534
    -1.2797705 ]]

  [[-1.5912344   0.20777252 -0.43285406 ... -0.1675356  -0.7102615
    -1.2166344 ]
   [-0.7868527  -1.6073718  -0.392333   ... -0.75894797 -0.4079647
    -1.2190759 ]
   [-0.8922362  -1.5154754  -0.5177639  ... -1.1458592   0.04225418
    -0.81918186]
   ...
   [-0.91656446 -1.0473183  -1.0032847  ... -1.8697616  -0.1359748
    -1.1439781 ]
   [-1.0975006   0.37157917 -0.00895183 ... -0.17786475 -0.4199428
    -0.605641

In [82]:
#아무래도 진짜..코드 위에선  outputs.last_hidden_state 로 써서 12번째 레이어의 값으로 KQV 계산했을 수 있다.
from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn

class BertWithRawAttention(BertModel):
    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, inputs_embeds=None, output_attentions=True, output_hidden_states=False, return_dict=True):
        # BERT 모델의 기본 동작 수행
        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=True,  # 어텐션 값 반환 활성화
            output_hidden_states=True,  # 히든 스테이트 반환 활성화
            return_dict=True,
        )

        # 모델 설정값 가져오기
        num_heads = self.config.num_attention_heads  # KLUE-BERT의 멀티헤드 개수 (12)
        hidden_dim = self.config.hidden_size  # BERT의 Hidden Dimension (768)
        head_dim = hidden_dim // num_heads  # 각 Head의 크기 (64)
        d_k = head_dim ** 0.5  # sqrt(d_k)

        # === 첫 번째 Transformer 레이어에서 Query (Q), Key (K) 직접 가져오기 ===
        first_layer = self.encoder.layer[0]  # 첫 번째 Transformer 레이어
        input_tensor = outputs.hidden_states[0]  # 첫 번째 레이어 입력 (Embedding 후 결과)

        # Query, Key 생성
        Q = first_layer.attention.self.query(input_tensor)  # (1, 10, 768)
        K = first_layer.attention.self.key(input_tensor)  # (1, 10, 768)

        # === Multi-Head 형태로 변환 ===
        # Query, Key의 shape을 (batch_size, num_heads, sequence_length, head_dim)로 변환
        Q = Q.view(1, 10, num_heads, head_dim).transpose(1, 2)  # (1, 12, 10, 64)
        K = K.view(1, 10, num_heads, head_dim).transpose(1, 2)  # (1, 12, 10, 64)

        # === Softmax 이전 Attention Score 계산 ===
        raw_attention = torch.matmul(Q, K.transpose(-2, -1)) / d_k  # (1, 12, 10, 10)

        return raw_attention, outputs.attentions[0]  # Softmax 이전 & 이후 값 반환

# === 모델 로드 ===
model_name = "klue/bert-base"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertWithRawAttention.from_pretrained(model_name)

# === 입력 문장 ===
text = "BERT 모델이 잘 동작하는지 확인합니다."
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=10)

# === 모델 실행 ===
model.eval()
with torch.no_grad():
    raw_attention, softmax_attention = model(**inputs)

# === 첫 번째 레이어의 Softmax 이전 Attention Score 확인 ===
layer_1_raw_attention = raw_attention.numpy()
print(f"Layer 1 Softmax 이전 Attention Score (Shape: {layer_1_raw_attention.shape}):")
print(layer_1_raw_attention)


Layer 1 Softmax 이전 Attention Score (Shape: (1, 12, 10, 10)):
[[[[ 3.479377    0.11714409  0.5856975  ...  0.49772462  2.0617602
     1.4053444 ]
   [ 1.6703308   3.3466518   5.793112   ...  2.3886065   1.5484276
     1.9099072 ]
   [ 3.3168168   4.502995    3.8073637  ...  1.6874914   2.6028967
     1.5656495 ]
   ...
   [ 1.6387088   1.7666457   2.1017816  ...  2.9609547   3.9101954
     4.537689  ]
   [ 3.3072975   1.2618201   2.365435   ...  4.3632503   3.529175
     5.7495446 ]
   [ 4.5703983   3.1162205   2.4864216  ...  4.4660215   6.618206
     5.1483626 ]]

  [[-1.0886638   0.02555725  0.311149   ... -0.46038562 -0.6803706
     0.9627847 ]
   [-1.7430696  -0.5790163   0.7301598  ... -0.7375137  -0.6899191
     0.97234195]
   [-0.43296996 -0.34776625 -0.61825126 ... -0.0585902  -1.3146526
     1.1253307 ]
   ...
   [-0.35978922  0.09165996  0.42240837 ... -0.33630806 -0.56914496
     0.99600965]
   [-0.96181583  0.22939461  0.1490941  ...  0.6880506  -0.33433583
     0.4417293 ]

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# 1. 토크나이저 로드
model_name = "klue/bert-base"  # 모델 이름 (학습에 사용된 것과 동일해야 함)
tokenizer = BertTokenizer.from_pretrained(model_name)

# 2. 사전 학습된 모델 로드
num_labels = len(label_names)  # 멀티라벨 분류를 위한 레이블 수
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# 저장된 가중치 로드
checkpoint_path = "/content/model/BERT_multilabel_model.pt"  # 저장된 모델 경로
model.load_state_dict(torch.load(checkpoint_path, map_location=torch.device('cpu')))

# 3. Attention 추출을 위해 모델 설정 변경
model.config.output_attentions = True  # Attention 점수 출력 활성화
model.eval()  # 평가 모드

# 4. 입력 텍스트
text = "ㅅ발 천주교도들은 너무 말이 많아"
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=10)

# 5. 모델 추론
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits  # 최종 예측 값 (softmax 이전)
    attentions = outputs.attentions  # Attention 점수

# 6. Softmax 이전의 값을 출력
print(f"Logits (Softmax 이전 값): {logits}")

# 7. Attention 점수를 출력 (각 Layer마다 Attention 점수가 포함됨)
for i, attention_layer in enumerate(attentions):
    print(f"Layer {i + 1} Attention Scores: {attention_layer.shape}")

# 각 Attention Layer의 Shape
for i, attention in enumerate(attentions):
    print(f"Layer {i + 1} Attention Shape: {attention.shape}")

# 첫 번째 레이어의 Attention 값을 numpy로 변환하여 확인
layer_1_attention = attentions[0].numpy()
print(f"Layer 1 Attention Scores (Shape: {layer_1_attention.shape}):")
print(layer_1_attention)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-76-ccee9f185cc6>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recomm

Logits (Softmax 이전 값): tensor([[-2.4690, -4.7337, -4.9942,  2.2562, -5.2708, -4.6522, -5.1170,  1.8131,
         -4.5230]])
Layer 1 Attention Scores: torch.Size([1, 12, 10, 10])
Layer 2 Attention Scores: torch.Size([1, 12, 10, 10])
Layer 3 Attention Scores: torch.Size([1, 12, 10, 10])
Layer 4 Attention Scores: torch.Size([1, 12, 10, 10])
Layer 5 Attention Scores: torch.Size([1, 12, 10, 10])
Layer 6 Attention Scores: torch.Size([1, 12, 10, 10])
Layer 7 Attention Scores: torch.Size([1, 12, 10, 10])
Layer 8 Attention Scores: torch.Size([1, 12, 10, 10])
Layer 9 Attention Scores: torch.Size([1, 12, 10, 10])
Layer 10 Attention Scores: torch.Size([1, 12, 10, 10])
Layer 11 Attention Scores: torch.Size([1, 12, 10, 10])
Layer 12 Attention Scores: torch.Size([1, 12, 10, 10])
Layer 1 Attention Shape: torch.Size([1, 12, 10, 10])
Layer 2 Attention Shape: torch.Size([1, 12, 10, 10])
Layer 3 Attention Shape: torch.Size([1, 12, 10, 10])
Layer 4 Attention Shape: torch.Size([1, 12, 10, 10])
Layer 5 Atten

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

class BertWithRawAttention(BertForSequenceClassification):
    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, inputs_embeds=None, output_attentions=True, output_hidden_states=True, return_dict=True):
        # ✅ output_hidden_states=True 추가!
        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=True,
            output_hidden_states=True,  # ✅ 여기를 추가해야 hidden_states가 출력됨!
            return_dict=True,
        )

        # Softmax 이후의 Attention Score
        attention_probs = outputs.attentions

        # Softmax 이전 값 (QK^T / sqrt(d_k)) 추출
        raw_attentions = []
        for layer in self.bert.encoder.layer:
            Q = layer.attention.self.query(outputs.hidden_states[-1])  # ✅ 이제 None이 아님!
            K = layer.attention.self.key(outputs.hidden_states[-1])
            d_k = Q.shape[-1] ** 0.5  # sqrt(d_k)

            # Softmax 이전의 Attention Score
            raw_attention = torch.matmul(Q, K.transpose(-2, -1)) / d_k
            raw_attentions.append(raw_attention)

        return raw_attentions, attention_probs, outputs.logits

# 모델 로드
model_name = "klue/bert-base"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertWithRawAttention.from_pretrained(model_name, num_labels=len(label_names))

# 입력 문장
texts = ["ㅅ발 천주교도들은 너무 말이 많아"]

inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=10)

# 모델 실행
model.eval()
with torch.no_grad():
    raw_attentions, softmax_attentions, logits = model(**inputs)

# Softmax 이전의 Attention Score 확인
print(f"Layer 1 Softmax 이전 Attention Score (Shape: {raw_attentions[0].shape}):")
print(raw_attentions[0].numpy())

# Softmax 이후의 Attention Score 확인
print(f"Layer 1 Softmax 이후 Attention Score (Shape: {softmax_attentions[0].shape}):")
print(softmax_attentions[0].numpy())

# 최종 분류 logits 출력
print(f"Logits (분류 모델의 Softmax 이전 값): {logits}")


Some weights of BertWithRawAttention were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Layer 1 Softmax 이전 Attention Score (Shape: torch.Size([1, 10, 10])):
[[[-1.25852823e+00  2.66215712e-01 -8.78310561e-01 -5.46610832e-01
   -2.88807482e-01  1.11422515e+00 -7.55868137e-01 -1.04048252e+00
   -6.23540282e-01 -4.86584216e-01]
  [-2.56121695e-01 -1.71548009e-01 -8.60907257e-01 -7.67546833e-01
   -9.95912910e-01 -1.00643361e+00 -4.07447457e-01 -1.75079393e+00
    1.79270849e-01  1.09233379e+00]
  [-2.95971274e-01 -5.97175062e-01 -5.01220524e-01 -7.32082844e-01
   -9.14117396e-01 -4.85211074e-01 -9.03017342e-01 -1.16841352e+00
   -1.58572093e-01  1.26348448e+00]
  [-1.37948975e-01 -2.04223856e-01  4.05856907e-01 -1.50166214e+00
    5.47514558e-02  1.44970313e-01 -1.13640618e+00 -2.22163677e+00
   -2.47296557e-01 -1.33462834e+00]
  [ 1.01324928e+00  1.64172506e+00  6.23836458e-01  6.27645195e-01
   -6.22797310e-01  9.15249646e-01  3.98019969e-01 -1.16255677e+00
   -6.03965938e-01 -1.14379549e+00]
  [ 1.11805654e+00  9.22619820e-01  1.66346371e+00  1.28638303e+00
    3.01247567

In [89]:
file_path = './model/attention_scores_2.txt'  # 읽고자 하는 파일의 경로

try:
    with open(file_path, 'r') as file:  # 파일을 읽기 모드로 열기
        content = file.read()  # 파일의 내용을 전체 읽기
    print(content)  # 읽은 내용 출력
except FileNotFoundError:
    print(f"The file {file_path} does not exist.")  # 파일이 존재하지 않을 경우 에러 메시지 출력

[[ 3.479377    0.11714409  0.5856975   0.6307755  -0.30920362  2.1947367
  -0.14835791  0.49772462  2.0617602   1.4053444 ]
 [ 1.6703308   3.3466518   5.793112    4.621073    3.812749    2.3419719
   1.8878778   2.3886065   1.5484276   1.9099072 ]
 [ 3.3168168   4.502995    3.8073637   5.2124023   2.3082643   3.0351076
   2.4797425   1.6874914   2.6028967   1.5656495 ]
 [ 2.5718498   4.652975    6.7969666   3.8015726   5.3321524   2.659658
   2.719986    2.416732    2.1785326   3.108758  ]
 [ 0.62522817  4.6419153   3.3077164   5.064213    3.339889    2.7762215
   1.873533    3.4980125   2.023627    2.5917144 ]
 [ 3.9795005   2.6142015   2.2757936   2.9215946   3.511932    3.220555
   3.0432897   2.8711824   4.3125715   4.3036885 ]
 [ 1.7959094   2.6852992   1.6750463   1.3191165   4.1721644   3.331105
   2.4741642   4.5505824   2.6400094   3.1753776 ]
 [ 1.6387088   1.7666457   2.1017816   2.1071794   3.422982    3.3852067
   4.2240024   2.9609547   3.9101954   4.537689  ]
 [ 3.307297

In [112]:
import pickle
import numpy as np

# 1. 파일 로드
file_path = "/./content/model/attention_scores.pkl"  # 파일 경로
with open(file_path, "rb") as f:
    attention_scores = pickle.load(f)

# 2. 각 최소 단위(10x10 행렬의 10개 값) 내림차순 정렬
sorted_attention_scores = np.zeros_like(attention_scores)  # 원래 크기의 배열 생성
for i in range(attention_scores.shape[0]):  # 첫 번째 차원 순회
    for j in range(attention_scores.shape[1]):  # 두 번째 차원 순회
        for k in range(attention_scores.shape[2]):  # 세 번째 차원 순회
            # 최소 단위(10개) 값을 내림차순으로 정렬
            sorted_attention_scores[i, j, k] = np.sort(attention_scores[i, j, k])[::-1]

# 3. 정렬된 데이터를 새로운 파일로 저장
sorted_file_path = "/./content/model/sorted_attention_scores.pkl"
with open(sorted_file_path, "wb") as f:
    pickle.dump(sorted_attention_scores, f)

print(f"정렬된 attention_scores가 {sorted_file_path}에 저장되었습니다!")



정렬된 attention_scores가 /./content/model/sorted_attention_scores.pkl에 저장되었습니다!


In [93]:
import pickle
import numpy as np

# ✅ 지수 표기법 억제 (출력 시 적용됨)
np.set_printoptions(suppress=True)

# 1. 파일 로드
file_path = "/./content/model/attention_scores.pkl"  # 파일 경로
with open(file_path, "rb") as f:
    attention_scores = pickle.load(f)

# ✅ dtype을 float32로 유지하여 값의 정밀도를 유지
attention_scores = attention_scores.astype(np.float32)

# 2. 원본 값을 유지하면서 내림차순 정렬 (argsort 사용)
sorted_attention_scores = np.zeros_like(attention_scores)  # 동일한 크기의 배열 생성
for i in range(attention_scores.shape[0]):  # 첫 번째 차원 순회
    for j in range(attention_scores.shape[1]):  # 두 번째 차원 순회
        for k in range(attention_scores.shape[2]):  # 세 번째 차원 순회
            # ✅ 원본 값의 순서를 유지하면서 내림차순 정렬
            indices = np.argsort(-attention_scores[i, j, k])  # 내림차순 정렬된 인덱스 가져오기
            sorted_attention_scores[i, j, k] = attention_scores[i, j, k][indices]  # 원본 값 유지

# 3. 정렬된 데이터를 새로운 파일로 저장
sorted_file_path = "/./content/model/sorted_attention_scores.pkl"
with open(sorted_file_path, "wb") as f:
    pickle.dump(sorted_attention_scores.astype(np.float32), f)  # ✅ 저장 시에도 float32 유지

print(f"정렬된 attention_scores가 {sorted_file_path}에 저장되었습니다! 🚀")



정렬된 attention_scores가 /./content/model/sorted_attention_scores.pkl에 저장되었습니다! 🚀


In [ ]:
outputs_2 = []

import pickle
with open("/./content/model/sorted_attention_scores.pkl", "rb") as f:
    outputs_2 = pickle.load(f)

print(outputs_2)

[[[[0.50276214 0.18352194 0.07385454 ... 0.02399363 0.01801627
    0.00693945]
   [0.28548524 0.15781423 0.12932299 ... 0.0462996  0.03935906
    0.03413377]
   [0.42965856 0.23049009 0.119009   ... 0.03273839 0.02393627
    0.01153364]
   ...
   [0.3043518  0.23911789 0.14382176 ... 0.02023179 0.00734722
    0.00598513]
   [0.42806664 0.27252406 0.0660226  ... 0.03525357 0.02467276
    0.0087873 ]
   [0.35068387 0.13904521 0.13328184 ... 0.01395757 0.00985229
    0.00765489]]

  [[0.2325552  0.13501358 0.11216996 ... 0.06796568 0.04669739
    0.01961427]
   [0.3069954  0.1544281  0.12369999 ... 0.0540519  0.03094326
    0.01976733]
   [0.3473453  0.17405728 0.11080742 ... 0.02729124 0.0272199
    0.01510452]
   ...
   [0.34108797 0.14767179 0.09490318 ... 0.05895856 0.02201105
    0.01425752]
   [0.16335478 0.15715772 0.1307724  ... 0.07702713 0.06241015
    0.02915837]
   [0.21095937 0.16749963 0.14411137 ... 0.04625707 0.04081224
    0.03196875]]

  [[0.20825657 0.19249158 0.1838356

In [94]:
pickle_file_path = "/./content/model/sorted_attention_scores.pkl"
text_file_path = "/./content/model/sorted_attention_scores_2.txt"
pickle_to_text(pickle_file_path, text_file_path)

In [2]:
# 파일을 열고 읽은 다음 내용을 출력하는 코드
file_path = "/./content/model/sorted_attention_scores_2.txt"  # 읽고자 하는 파일의 경로

try:
    with open(file_path, 'r') as file:  # 파일을 읽기 모드로 열기
        content = file.read()  # 파일의 내용을 전체 읽기
    print(content)  # 읽은 내용 출력
except FileNotFoundError:
    print(f"The file {file_path} does not exist.")  # 파일이 존재하지 않을 경우 에러 메시지 출력

[[ 3.479377    2.1947362   2.06176     1.4053445   0.6307752   0.58569705
   0.49772385  0.1171443  -0.14835815 -0.30920365]
 [ 5.7931113   4.6210723   3.8127494   3.3466516   2.388606    2.3419714
   1.9099065   1.8878773   1.6703308   1.5484267 ]
 [ 5.212402    4.502995    3.8073645   3.3168163   3.035107    2.6028965
   2.4797413   2.3082643   1.6874913   1.5656486 ]
 [ 6.7969666   5.332153    4.652974    3.8015728   3.1087575   2.7199864
   2.6596584   2.57185     2.4167328   2.178532  ]
 [ 5.0642123   4.6419163   3.4980135   3.33989     3.3077157   2.776222
   2.591714    2.023627    1.8735324   0.62522924]
 [ 4.312572    4.3036885   3.9794996   3.5119328   3.2205546   3.0432897
   2.9215946   2.8711832   2.6142018   2.2757945 ]
 [ 4.5505824   4.1721644   3.3311045   3.1753774   2.6852996   2.6400094
   2.474163    1.7959092   1.6750463   1.319116  ]
 [ 4.5376897   4.2240024   3.9101956   3.422982    3.3852065   2.9609554
   2.1071799   2.1017816   1.7666456   1.6387084 ]
 [ 5.749

In [ ]:
import numpy as np
import pickle

# 1. 파일에서 attention_scores 로드
file_path = "./model/attention_scores.pkl"
with open(file_path, "rb") as f:
    attention_scores = pickle.load(f)

# 2. Softmax 함수 정의
def softmax(x):
    exp_x = np.exp(x - np.max(x))  # Overflow 방지를 위해 x에서 최대값을 뺌
    return exp_x / exp_x.sum()

# 3. attention_scores에 Softmax 연산 적용
softmax_values = np.zeros_like(attention_scores)  # 원래 크기의 배열 생성
for i in range(attention_scores.shape[0]):  # 첫 번째 차원 순회
    for j in range(attention_scores.shape[1]):  # 두 번째 차원 순회
        for k in range(attention_scores.shape[2]):  # 세 번째 차원 순회
            softmax_values[i, j, k] = softmax(attention_scores[i, j, k])  # 최소 단위(10개)에 Softmax 적용

# 4. Softmax 결과 확인
print("Softmax 값:")
print(softmax_values)

# 5. Softmax 값의 합 (확률 분포인지 확인)
print(f"Softmax 값의 합 (각 10개 묶음에서):")
for i in range(softmax_values.shape[0]):
    for j in range(softmax_values.shape[1]):
        for k in range(softmax_values.shape[2]):
            print(f"Sum of softmax values in [{i}, {j}, {k}]: {softmax_values[i, j, k].sum()}")

# 6. Softmax 값을 새로운 파일로 저장
softmax_file_path = "./model/softmax_attention_scores.pkl"
with open(softmax_file_path, "wb") as f:
    pickle.dump(softmax_values, f)

print(f"Softmax attention_scores가 {softmax_file_path}에 저장되었습니다!")


Softmax 값:
[[[[0.14792055 0.09109741 0.09164357 ... 0.09258842 0.09229336
    0.09513694]
   [0.12004586 0.10268968 0.10120098 ... 0.09385469 0.09336554
    0.09450836]
   [0.09188308 0.11296417 0.10104739 ... 0.09287033 0.09284727
    0.09269542]
   ...
   [0.09724706 0.09072162 0.09059814 ... 0.09612775 0.12209511
    0.1143846 ]
   [0.09323013 0.09044815 0.09189644 ... 0.11774409 0.09348241
    0.13755944]
   [0.10347308 0.09130668 0.09073301 ... 0.10249271 0.0964129
    0.10287845]]

  [[0.09785708 0.1010711  0.09990909 ... 0.1006773  0.09957507
    0.11400125]
   [0.09519613 0.09198768 0.10206269 ... 0.09563526 0.09920019
    0.12259457]
   [0.0963392  0.09690202 0.09143765 ... 0.10719077 0.10062093
    0.12747219]
   ...
   [0.09908457 0.09758333 0.09906299 ... 0.0961072  0.0978529
    0.12674263]
   [0.09308667 0.10304288 0.09866013 ... 0.10144265 0.09808288
    0.10579788]
   [0.09461471 0.1043413  0.0984853  ... 0.10681042 0.09746661
    0.11155472]]

  [[0.10933199 0.09290536

In [ ]:
import pickle

# 파일 경로
file_path = "./model/attention_scores.pkl"

# 파일에서 데이터 로드
with open(file_path, "rb") as f:
    attention_scores = pickle.load(f)

# 데이터 형태 확인
print(f"Shape of attention_scores: {np.array(attention_scores).shape}")


Shape of attention_scores: (1, 12, 10, 10)


In [ ]:
import pickle
import numpy as np

# 1. 파일 로드
file_path = "./model/attention_scores.pkl"  # 기존 파일 경로
with open(file_path, "rb") as f:
    attention_scores = pickle.load(f)

# 2. 정렬 및 인덱스 저장
sorted_attention_scores = np.zeros_like(attention_scores)
original_indices = np.zeros_like(attention_scores, dtype=int)

for i in range(attention_scores.shape[0]):  # 첫 번째 차원 순회
    for j in range(attention_scores.shape[1]):  # 두 번째 차원 순회
        for k in range(attention_scores.shape[2]):  # 세 번째 차원 순회
            # 원래 인덱스와 정렬된 배열 저장
            sorted_indices = np.argsort(attention_scores[i, j, k])[::-1]  # 내림차순 인덱스
            sorted_attention_scores[i, j, k] = attention_scores[i, j, k][sorted_indices]
            original_indices[i, j, k] = sorted_indices  # 원래의 순서 저장

# 3. 정렬된 데이터를 새로운 파일로 저장
sorted_file_path = "./model/sorted_attention_scores.pkl"
with open(sorted_file_path, "wb") as f:
    pickle.dump(sorted_attention_scores, f)

# 4. 원래 인덱스 데이터를 새로운 파일로 저장
indices_file_path = "./model/original_indices.pkl"
with open(indices_file_path, "wb") as f:
    pickle.dump(original_indices, f)

print(f"정렬된 attention_scores와 원래 인덱스가 각각 {sorted_file_path}, {indices_file_path}에 저장되었습니다!")


정렬된 attention_scores와 원래 인덱스가 각각 ./model/sorted_attention_scores.pkl, ./model/original_indices.pkl에 저장되었습니다!


In [ ]:
import numpy as np
import pickle

# 1. 파일에서 attention_scores 로드
file_path = "./model/sorted_attention_scores.pkl"
with open(file_path, "rb") as f:
    attention_scores = pickle.load(f)

# 2. Softmax 함수 정의
def softmax(x):
    exp_x = np.exp(x - np.max(x))  # Overflow 방지를 위해 x에서 최대값을 뺌
    return exp_x / exp_x.sum()

# 3. attention_scores에 Softmax 연산 적용
softmax_values = np.zeros_like(attention_scores)  # 원래 크기의 배열 생성
for i in range(attention_scores.shape[0]):  # 첫 번째 차원 순회
    for j in range(attention_scores.shape[1]):  # 두 번째 차원 순회
        for k in range(attention_scores.shape[2]):  # 세 번째 차원 순회
            softmax_values[i, j, k] = softmax(attention_scores[i, j, k])  # 최소 단위(10개)에 Softmax 적용

# 4. Softmax 결과 확인
print("Softmax 값:")
print(softmax_values)

# 5. Softmax 값의 합 (확률 분포인지 확인)
print(f"Softmax 값의 합 (각 10개 묶음에서):")
for i in range(softmax_values.shape[0]):
    for j in range(softmax_values.shape[1]):
        for k in range(softmax_values.shape[2]):
            print(f"Sum of softmax values in [{i}, {j}, {k}]: {softmax_values[i, j, k].sum()}")

# 6. Softmax 값을 새로운 파일로 저장
softmax_file_path2 = "./model/softmax_sorted_attention_scores.pkl"
with open(softmax_file_path2, "wb") as f:
    pickle.dump(softmax_values, f)

print(f"Softmax sorted_attention_scores가 {softmax_file_path2}에 저장되었습니다!")

Softmax 값:
[[[[0.14792053 0.107494   0.09632882 ... 0.09164356 0.0910974
    0.0900939 ]
   [0.12004589 0.10565753 0.10268968 ... 0.09450835 0.0938547
    0.09336554]
   [0.13786004 0.11296419 0.1010474  ... 0.09269542 0.09188308
    0.09075053]
   ...
   [0.12209511 0.1143846  0.10398747 ... 0.0918981  0.09072163
    0.09059815]
   [0.13755941 0.11774407 0.095776   ... 0.09287394 0.09189644
    0.09044816]
   [0.12786184 0.10347308 0.10287845 ... 0.09130669 0.09093262
    0.09073301]]

  [[0.11400125 0.1034065  0.1010711  ... 0.09670064 0.09466569
    0.09213626]
   [0.12259459 0.10524758 0.10206271 ... 0.09519614 0.0930215
    0.09198768]
   [0.1274722  0.10719078 0.10062095 ... 0.0925588  0.09255221
    0.09143765]
   ...
   [0.12674265 0.10445354 0.09908457 ... 0.09558626 0.09211903
    0.09140754]
   [0.10645555 0.10579788 0.10304288 ... 0.097651   0.09623402
    0.09308667]
   [0.11155473 0.10681042 0.10434131 ... 0.09461471 0.09410095
    0.09327244]]

  [[0.11106926 0.109332   

In [ ]:
pickle_file_path = "/./content/model/softmax_attention_scores.pkl"
text_file_path = "/./content/model/softmax_attention_scores_2.txt"
pickle_to_text(pickle_file_path, text_file_path)

In [ ]:
pickle_file_path = "/./content/model/softmax_sorted_attention_scores.pkl"
text_file_path = "/./content/model/softmax_sorted_attention_scores_2.txt"
pickle_to_text(pickle_file_path, text_file_path)

In [ ]:
pickle_file_path = "/./content/model/restored_softmax.pkl"
text_file_path = "/./content/model/restored_softmax_2.txt"
pickle_to_text(pickle_file_path, text_file_path)

In [ ]:
try:
    with open("/./content/model/softmax_sorted_attention_scores_2.txt", 'r') as file:  # 파일을 읽기 모드로 열기
        content = file.read()  # 파일의 내용을 전체 읽기
    print(content)  # 읽은 내용 출력
except FileNotFoundError:
    print(f"The file {file_path} does not exist.")  # 파일이 존재하지 않을 경우 에러 메시지 출력

[[0.14792053 0.10749399 0.09632882 0.0954031  0.09513693 0.09258841
  0.09229334 0.09164356 0.0910974  0.0900939 ]
 [0.12004588 0.10565753 0.10268969 0.10120098 0.09709173 0.0958351
  0.09575053 0.09450836 0.0938547  0.09336555]
 [0.13786007 0.11296417 0.10104739 0.09357624 0.09350549 0.09287033
  0.09284727 0.09269542 0.09188308 0.09075052]
 [0.11074849 0.10814226 0.10393477 0.10047178 0.10030901 0.09913348
  0.09670201 0.09443171 0.09372903 0.09239744]
 [0.1193184  0.10324923 0.10189565 0.09932904 0.09908514 0.09784093
  0.09755905 0.0950703  0.09426644 0.09238588]
 [0.1328674  0.10326969 0.09898279 0.09854479 0.09625035 0.09615592
  0.0958468  0.09484941 0.09185758 0.0913753 ]
 [0.11864013 0.1089671  0.10785589 0.10248473 0.09685351 0.09558748
  0.09497281 0.09192704 0.0917357  0.0909756 ]
 [0.12209511 0.1143846  0.10398746 0.09853819 0.09724706 0.09612775
  0.09440193 0.09189809 0.09072162 0.09059814]
 [0.13755944 0.11774409 0.09577599 0.0939422  0.09348241 0.09323013
  0.09304722 

In [ ]:
try:
    with open( "/./content/model/softmax_attention_scores_2.txt", 'r') as file:  # 파일을 읽기 모드로 열기
        content = file.read()  # 파일의 내용을 전체 읽기
    print(content)  # 읽은 내용 출력
except FileNotFoundError:
    print(f"The file {file_path} does not exist.")  # 파일이 존재하지 않을 경우 에러 메시지 출력

[[0.14792055 0.09109741 0.09164357 0.09009392 0.09632883 0.09540311
  0.107494   0.09258842 0.09229336 0.09513694]
 [0.12004586 0.10268968 0.10120098 0.09575053 0.09583508 0.09709172
  0.10565752 0.09385469 0.09336554 0.09450836]
 [0.09188308 0.11296417 0.10104739 0.13786007 0.09357624 0.09350549
  0.09075052 0.09287033 0.09284727 0.09269542]
 [0.11074849 0.10814226 0.10393477 0.09670201 0.09913348 0.10047178
  0.10030901 0.09443171 0.09239744 0.09372903]
 [0.09755905 0.09238588 0.09932904 0.10189565 0.09426644 0.1193184
  0.09908514 0.09784093 0.0950703  0.10324923]
 [0.09625035 0.09185757 0.09137529 0.09854478 0.09584679 0.0948494
  0.09615591 0.13286738 0.09898278 0.10326968]
 [0.10248473 0.0909756  0.09192704 0.0917357  0.09685351 0.10785589
  0.09497281 0.1089671  0.09558748 0.11864013]
 [0.09724706 0.09072162 0.09059814 0.09189809 0.09440193 0.09853819
  0.10398746 0.09612775 0.12209511 0.1143846 ]
 [0.09323013 0.09044815 0.09189644 0.09287393 0.09304722 0.0939422
  0.09577599 0.

In [ ]:
import pickle
import numpy as np

# 1. 정렬된 데이터와 원래 인덱스 로드
sorted_file_path3 = "./model/sorted_attention_scores.pkl"
indices_file_path3 = "./model/original_indices.pkl"
restored_file_path3 = "./model/restored_softmax.pkl"

with open(sorted_file_path3, "rb") as f:
    sorted_attention_scores = pickle.load(f)

with open(indices_file_path3, "rb") as f:
    original_indices = pickle.load(f)

# 2. 원래 순서로 복원
restored_attention_scores = np.zeros_like(sorted_attention_scores)
for i in range(sorted_attention_scores.shape[0]):  # 첫 번째 차원 순회
    for j in range(sorted_attention_scores.shape[1]):  # 두 번째 차원 순회
        for k in range(sorted_attention_scores.shape[2]):  # 세 번째 차원 순회
            # 원래 순서로 복원
            inverse_indices = np.argsort(original_indices[i, j, k])  # 원래 순서 찾기
            restored_attention_scores[i, j, k] = sorted_attention_scores[i, j, k][inverse_indices]

# 3. 복원된 결과를 새로운 파일에 저장
with open(restored_file_path3, "wb") as f:
    pickle.dump(restored_attention_scores, f)

print(f"복원된 softmax attention_scores가 {restored_file_path3}에 저장되었습니다!")


복원된 softmax attention_scores가 ./model/restored_softmax.pkl에 저장되었습니다!


In [ ]:
try:
    with open("/./content/model/restored_softmax_2.txt", 'r') as file:  # 파일을 읽기 모드로 열기
        content = file.read()  # 파일의 내용을 전체 읽기
    print(content)  # 읽은 내용 출력
except FileNotFoundError:
    print(f"The file {file_path} does not exist.")  # 파일이 존재하지 않을 경우 에러 메시지 출력

[[0.14792053 0.0910974  0.09164356 0.0900939  0.09632882 0.0954031
  0.10749399 0.09258841 0.09229334 0.09513693]
 [0.12004588 0.10268969 0.10120098 0.09575053 0.0958351  0.09709173
  0.10565753 0.0938547  0.09336555 0.09450836]
 [0.09188308 0.11296417 0.10104739 0.13786007 0.09357624 0.09350549
  0.09075052 0.09287033 0.09284727 0.09269542]
 [0.11074849 0.10814226 0.10393477 0.09670201 0.09913348 0.10047178
  0.10030901 0.09443171 0.09239744 0.09372903]
 [0.09755905 0.09238588 0.09932904 0.10189565 0.09426644 0.1193184
  0.09908514 0.09784093 0.0950703  0.10324923]
 [0.09625035 0.09185758 0.0913753  0.09854479 0.0958468  0.09484941
  0.09615592 0.1328674  0.09898279 0.10326969]
 [0.10248473 0.0909756  0.09192704 0.0917357  0.09685351 0.10785589
  0.09497281 0.1089671  0.09558748 0.11864013]
 [0.09724706 0.09072162 0.09059814 0.09189809 0.09440193 0.09853819
  0.10398746 0.09612775 0.12209511 0.1143846 ]
 [0.09323013 0.09044815 0.09189644 0.09287393 0.09304722 0.0939422
  0.09577599 0.

In [ ]:
import numpy as np

def parse_text_file(file_path):
    with open(file_path, "r") as f:
        data = f.read()

    # 데이터 전처리: 불필요한 문자 제거 및 정리
    data = data.replace("\n", " ")  # 줄바꿈 제거
    data = data.replace("[", "").replace("]", "")  # 대괄호 제거
    data = data.split()  # 공백 기준으로 나눔

    # 문자열을 float으로 변환
    try:
        data_list = [float(value) for value in data]
        return np.array(data_list)  # numpy 배열로 변환
    except ValueError as e:
        print(f"데이터 변환 중 오류 발생: {e}")
        raise

# 두 파일의 데이터 읽기
try:
    restored_data = parse_text_file('./model/restored_softmax_2.txt')
    original_data = parse_text_file('./model/attention_scores_2.txt')

    # 데이터 형태 확인
    print(f"Restored data shape: {restored_data.shape}")
    print(f"Original data shape: {original_data.shape}")

    # 데이터 값 하나하나 비교
    if np.array_equal(restored_data, original_data):
        print("두 데이터는 형태와 값이 모두 동일합니다.")
    else:
        print("데이터에 차이가 있습니다.")

        # 값이 다를 경우, 차이를 확인
        differences = np.where(restored_data != original_data)
        print(f"값이 다른 위치: {differences}")
        print(f"Restored data at differences: {restored_data[differences]}")
        print(f"Original data at differences: {original_data[differences]}")

except Exception as e:
    print(f"파일 처리 중 오류 발생: {e}")


Restored data shape: (1200,)
Original data shape: (1200,)
두 데이터는 형태와 값이 모두 동일합니다.


In [ ]:
import numpy as np

# 10진수 소수를 32비트 fixed-point binary로 변환하는 함수
def float_to_fixed_point(value, frac_bits=13):
    """
    소수점 값을 32비트 fixed-point binary로 변환.
    - 31~13 비트: 0
    - 12~0 비트: 소수점 값
    """
    max_value = 2**frac_bits - 1  # 13비트 최대 값
    fixed_point = int(value * max_value)  # 소수점 값을 frac 부분으로 변환
    fixed_binary = format(fixed_point, f'032b')  # 32비트 binary 표현
    return fixed_binary

# txt 파일 처리 함수
def process_txt_file(input_file, output_file):
    with open(input_file, 'r') as f:
        # 파일에서 데이터를 읽어옵니다
        data = f.readlines()

    # 변환된 데이터를 저장할 리스트
    converted_data = []

    for line in data:
        # [[ 및 ]] 제거 (데이터 전처리)
        line = line.strip().replace("[", "").replace("]", "")
        # 각 줄에서 숫자 데이터 추출
        try:
            numbers = list(map(float, line.split()))
        except ValueError:
            print(f"라인 변환 중 오류 발생: {line}")
            continue
        # 32비트 fixed-point 형식으로 변환
        converted_numbers = [float_to_fixed_point(num) for num in numbers]
        # 변환된 줄을 다시 저장
        converted_data.append(" ".join(converted_numbers))

    # 결과를 출력 파일로 저장
    with open(output_file, 'w') as f:
        f.write("\n".join(converted_data))

# 입력 파일과 출력 파일 경로
input_file = './model/sorted_attention_scores_2.txt'  # 원본 파일 경로
output_file = './model/output.txt'  # 변환된 파일 저장 경로

# txt 파일 변환 수행
process_txt_file(input_file, output_file)
print(f"파일이 성공적으로 변환되어 {output_file}에 저장되었습니다.")


파일이 성공적으로 변환되어 ./model/output.txt에 저장되었습니다.


In [100]:
import numpy as np

def float_to_fixed_point(value, int_bits=7, frac_bits=13):
    """
    실수를 32비트 Fixed-Point 바이너리로 변환하는 함수.
    - 31~20 비트: 항상 0 (상위 12비트)
    - 19~13 비트: 정수부 (7비트, Signed, 2의 보수)
    - 12~0 비트: 소수부 (13비트, 항상 양수)

    음수의 경우 정수부를 내림하여 변환하고, 남은 값을 소수부로 변환.
    """
    max_int_value = 2**(int_bits - 1) - 1  # 7비트 최대값 (63)
    min_int_value = -2**(int_bits - 1)     # 7비트 최소값 (-64)
    max_frac_value = 2**frac_bits           # 13비트 정밀도 (8192)

    # ✅ 정수부 조정 (floor 적용하여 내림)
    int_part = int(np.floor(value))
    frac_part = abs(value - int_part)

    # ✅ 정수부 범위 확인
    if int_part < min_int_value or int_part > max_int_value:
        raise ValueError(f"정수부 {int_part}가 {min_int_value} ~ {max_int_value} 범위를 벗어남!")

    # ✅ 2의 보수 변환 (정수부 7비트)
    if int_part < 0:
        int_binary = format((1 << int_bits) + int_part, f'0{int_bits}b')  # 7비트 2의 보수 변환
    else:
        int_binary = format(int_part, f'0{int_bits}b')  # 7비트 양수 표현

    # ✅ 앞 12비트는 항상 0 유지
    int_binary = "0" * 12 + int_binary

    # ✅ 소수부 변환 (13비트, 항상 양수)
    frac_binary = format(int(round(frac_part * max_frac_value)), f'0{frac_bits}b')

    # ✅ 최종 32비트 바이너리 생성
    fixed_binary = int_binary + frac_binary
    return fixed_binary

def process_txt_file(input_file, output_file):
    """
    TXT 파일을 읽어서 Fixed-Point 변환 후 새로운 파일에 저장하는 함수.
    """
    with open(input_file, 'r') as f:
        data = f.readlines()

    converted_data = []

    for line in data:
        line = line.strip().replace("[", "").replace("]", "")
        try:
            numbers = list(map(float, line.split()))  # 파일에서 숫자 읽어오기
        except ValueError:
            print(f"⚠️ 변환 오류 발생: {line}")
            continue

        # ✅ 32비트 Fixed-Point 변환 수행
        converted_numbers = [float_to_fixed_point(num) for num in numbers]
        converted_data.append(" ".join(converted_numbers))

    # 결과를 출력 파일로 저장
    with open(output_file, 'w') as f:
        f.write("\n".join(converted_data))

# ✅ 입력 파일과 출력 파일 경로 설정
input_file = './model/sorted_attention_scores_2.txt'  # 원본 파일 경로
output_file = './model/output.txt'  # 변환된 파일 저장 경로

# ✅ TXT 파일 변환 실행
process_txt_file(input_file, output_file)
print(f"✅ 파일이 성공적으로 변환되어 {output_file}에 저장되었습니다! 🚀")


✅ 파일이 성공적으로 변환되어 ./model/output.txt에 저장되었습니다! 🚀


In [ ]:
try:
    with open("/./content/model/output.txt", 'r') as file:  # 파일을 읽기 모드로 열기
        content = file.read()  # 파일의 내용을 전체 읽기
    print(content)  # 읽은 내용 출력
except FileNotFoundError:
    print(f"The file {file_path} does not exist.")  # 파일이 존재하지 않을 경우 에러 메시지 출력

00000000000000000001000000010110 00000000000000000000010111011111 00000000000000000000001001011100 00000000000000000000001000001101 00000000000000000000000111110110 00000000000000000000000100011000
00000000000000000000000011111110 00000000000000000000000011000100 00000000000000000000000010010011 00000000000000000000000000111000
00000000000000000000100100100010 00000000000000000000010100001100 00000000000000000000010000100011 00000000000000000000001110101011 00000000000000000000001001011000 00000000000000000000000111101101
00000000000000000000000111100110 00000000000000000000000101111011 00000000000000000000000101000010 00000000000000000000000100010111
00000000000000000000110110111111 00000000000000000000011101011111 00000000000000000000001111001110 00000000000000000000000101011001 00000000000000000000000101010011 00000000000000000000000100011011
00000000000000000000000100011001 00000000000000000000000100001100 00000000000000000000000011000100 00000000000000000000000001011110
0000000000

In [ ]:
def generate_code_from_txt(input_file, output_file):
    # 템플릿 형식
    template = "Xil_Out32(XPAR_AXI_BRAM_CTRL_0_S_AXI_BASEADDR + (4 * {index}), 0b{binary_value});"

    # txt 파일 읽기 및 값 분리
    with open(input_file, 'r') as f:
        content = f.read()  # 파일 전체 내용을 읽음
        binary_values = content.split()  # 스페이스와 줄바꿈을 기준으로 값을 분리

    # 새로운 파일 작성
    with open(output_file, 'w') as f:
        for i, binary_value in enumerate(binary_values):
            # 줄 생성
            line = template.format(index=i + 9, binary_value=binary_value)  # index는 9부터 시작
            f.write(line + "\n")  # 줄바꿈 포함

# 입력 파일과 출력 파일 경로 설정
input_txt_file = './model/output.txt'  # 입력 파일 경로 (값들이 스페이스/줄바꿈으로 구별됨)
output_code_file = './model/generated_code.txt'  # 생성할 코드 파일 경로

# 코드 생성 함수 호출
generate_code_from_txt(input_txt_file, output_code_file)
print(f"코드가 성공적으로 생성되어 {output_code_file}에 저장되었습니다.")


코드가 성공적으로 생성되어 ./model/generated_code.txt에 저장되었습니다.


In [ ]:
def convert_hex_to_16bit_binary(input_file, output_file):
    with open(input_file, 'r') as f:
        # 파일 내용 읽기 및 분리 (스페이스 및 줄바꿈 기준)
        hex_values = f.read().split()

    # 16진수 -> 16비트 2진수 변환
    binary_values = [format(int(hex_value, 16), '016b') for hex_value in hex_values]

    # 결과를 새 파일에 저장
    with open(output_file, 'w') as f:
        for binary_value in binary_values:
            f.write(binary_value + "\n")  # 각 값을 줄바꿈으로 저장

# 입력 및 출력 파일 경로
input_txt_file = './model/cordic_result.txt'  # 16진수 값이 저장된 입력 파일
output_txt_file = './model/binary_values.txt'  # 변환된 16비트 2진수를 저장할 출력 파일

# 함수 호출
convert_hex_to_16bit_binary(input_txt_file, output_txt_file)
print(f"16진수 값이 16비트 2진수로 변환되어 {output_txt_file}에 저장되었습니다.")


16진수 값이 16비트 2진수로 변환되어 ./model/binary_values.txt에 저장되었습니다.


In [ ]:
def convert_binary_to_decimal(input_file, output_file):
    def binary_to_decimal(binary_str):
        # 16비트 중 앞 3비트는 정수부, 뒤 13비트는 소수부
        int_part = int(binary_str[:3], 2)  # 정수부
        frac_part = int(binary_str[3:], 2) / (2 ** 13)  # 소수부를 2^13으로 나눔
        return int_part + frac_part  # 정수부와 소수부 합산

    with open(input_file, 'r') as f:
        # 파일에서 16비트 바이너리 값 읽기
        binary_values = f.read().splitlines()

    # 16비트 이진수를 10진수로 변환
    decimal_values = [binary_to_decimal(binary) for binary in binary_values]

    # 결과를 새 파일에 저장
    with open(output_file, 'w') as f:
        for decimal in decimal_values:
            f.write(f"{decimal:.10f}\n")  # 소수점 10자리까지 출력

# 입력 및 출력 파일 경로
input_txt_file = './model/binary_values.txt'  # 16비트 바이너리 입력 파일
output_txt_file = './model/cordic_dec_val.txt'  # 변환된 10진수 값을 저장할 출력 파일

# 함수 호출
convert_binary_to_decimal(input_txt_file, output_txt_file)
print(f"16비트 바이너리 값이 10진수 소수점 값으로 변환되어 {output_txt_file}에 저장되었습니다.")


16비트 바이너리 값이 10진수 소수점 값으로 변환되어 ./model/cordic_dec_val.txt에 저장되었습니다.


In [ ]:
def decimal_to_fixed_point_hex(input_file, output_file):
    def float_to_fixed_point(value):
        """
        10진수 소수를 16비트 고정소수점 표현으로 변환:
        - 앞 3비트: 정수부 (0~7)
        - 뒤 13비트: 소수부
        """
        # 정수부: value의 정수 부분 (0~7 사이 값)
        int_part = int(value)
        if int_part > 7:
            raise ValueError(f"정수부가 3비트를 초과했습니다: {value}")

        # 소수부: value의 소수 부분을 13비트로 표현
        frac_part = value - int_part
        frac_binary = int(round(frac_part * (2 ** 13)))  # 소수부를 2^13로 스케일링

        # 16비트 바이너리 표현
        binary_value = f"{int_part:03b}{frac_binary:013b}"
        return binary_value

    def binary_to_hex(binary_str):
        """
        16비트 바이너리를 16진수(hex)로 변환.
        """
        return f"0x{int(binary_str, 2):04X}"

    with open(input_file, 'r') as f:
        # 파일에서 10진수 소수점 값 읽기 (불필요한 문자 제거)
        raw_data = f.read()
        cleaned_data = raw_data.replace("[", "").replace("]", "").replace(",", "").strip()
        decimal_values = [float(value) for value in cleaned_data.split()]

    # 10진수를 16비트 고정소수점 -> 16진수로 변환
    hex_values = []
    for value in decimal_values:
        binary_value = float_to_fixed_point(value)  # 16비트 바이너리 변환
        hex_value = binary_to_hex(binary_value)  # 16진수 변환
        hex_values.append(hex_value)

    # 결과를 새 파일에 저장
    with open(output_file, 'w') as f:
        for hex_value in hex_values:
            f.write(f"{hex_value}\n")

# 입력 및 출력 파일 경로
input_txt_file = './model/softmax_sorted_attention_scores_2.txt'  # 입력 파일
output_txt_file = './model/converted_fixed_point_hex.txt'  # 변환된 16진수 값을 저장할 출력 파일

# 함수 호출
decimal_to_fixed_point_hex(input_txt_file, output_txt_file)
print(f"소수점 값이 16비트 16진수 값으로 변환되어 {output_txt_file}에 저장되었습니다.")


소수점 값이 16비트 16진수 값으로 변환되어 ./model/converted_fixed_point_hex.txt에 저장되었습니다.


In [ ]:
import numpy as np

def calculate_error_metrics(file1, file2, num_values=463):
    """
    두 파일에서 첫 번째 파일의 num_values만큼 데이터를 비교하여 1대1 오차율 및 평균 오차율 계산.

    Args:
    file1: 첫 번째 텍스트 파일 경로
    file2: 두 번째 텍스트 파일 경로
    num_values: 비교할 데이터 개수 (기본값: 463)

    Returns:
    각 값의 오차율 리스트 및 전체 평균 오차율
    """
    # 파일에서 데이터를 읽어오기
    def read_file(file_path):
        with open(file_path, 'r') as f:
            raw_data = f.read()
            # 불필요한 문자 제거 및 숫자 리스트로 변환
            cleaned_data = raw_data.replace("[", "").replace("]", "").replace(",", "").strip()
            return np.array([float(value) for value in cleaned_data.split()])

    # 두 파일의 데이터를 읽어오기
    values1 = read_file(file1)
    values2 = read_file(file2)

    # 데이터 길이 제한
    values1 = values1[:num_values]
    values2 = values2[:num_values]

    # 데이터 길이 확인 (463개 값이 맞는지)
    if len(values1) != len(values2):
        raise ValueError(f"비교할 데이터 길이가 다릅니다: {len(values1)} vs {len(values2)}")

    # 1대1 오차율 계산: |값1 - 값2| / 값1 * 100
    error_rates = np.abs(values1 - values2) / np.abs(values1) * 100

    # 전체 평균 오차율 계산
    mean_error_rate = np.mean(error_rates)

    return error_rates, mean_error_rate

# 파일 경로 설정
file1_path = './model/softmax_sorted_attention_scores_2.txt'  # 첫 번째 파일
file2_path = './model/cordic_dec_val.txt'  # 두 번째 파일

# 함수 호출
error_rates, mean_error_rate = calculate_error_metrics(file1_path, file2_path, num_values=463)

# 결과 출력
print(f"각 값의 오차율 (퍼센트):\n{error_rates}")
print(f"전체 평균 오차율 (퍼센트): {mean_error_rate:.6f}")


각 값의 오차율 (퍼센트):
[ 8.60191699  8.56347219  8.9813711   9.01522718  8.2937444   9.16508751
  9.51409798 10.29058539 10.95181509 12.18763845  9.41453822  9.52617972
  9.83866032  9.52449838  9.13086224  9.54281761  9.63956941 10.04731984
  9.77323863 10.34836172  9.44353868  9.35780286  8.72450078  9.05629591
  9.13881228  9.8852359   9.91252753 10.09258192 11.06591246 12.45199472
 10.663906    9.71874717 10.4020173  10.31936805  9.5248385   9.83845806
  9.57066063 10.136645   10.96232    12.56145863 10.0816439  10.18922969
 10.69458686 10.60539919  9.89207746 10.29142528 10.61010461 10.42404284
 10.32972731 12.57555533  9.88104141  9.22177712 10.00571897  9.50366448
 10.08483835  9.17734417  9.52944574  9.65163424 12.15988383 12.75186519
  9.06472473  8.88823754  9.10464065  9.10543083  9.39927953  8.80494621
  9.50913867 12.07512371 12.30888716 13.24722651  8.37798402  8.8535684
  8.46784598  9.51097671  8.95653935  9.2093156   9.13690409 11.04775964
 12.48781707 12.64113141  8.97280222

In [ ]:
import pickle

def txt_to_pkl(input_txt_file, output_pkl_file):
    # txt 파일 읽기
    with open(input_txt_file, 'r') as f:
        # 모든 줄에서 숫자 데이터를 추출
        data = [float(value) for line in f for value in line.split()]

    # pkl 파일로 저장
    with open(output_pkl_file, 'wb') as f:
        pickle.dump(data, f)

    print(f"TXT 파일이 {output_pkl_file}로 변환되었습니다!")

# 입력 txt 파일 경로와 출력 pkl 파일 경로
input_txt_file = './model/cordic_dec_val.txt'
output_pkl_file = './model/cordic_dec_val.pkl'

# 변환 함수 호출
txt_to_pkl(input_txt_file, output_pkl_file)


TXT 파일이 ./model/cordic_dec_val.pkl로 변환되었습니다!
